In [1]:
#| default_exp handlers.helcom

# HELCOM

> This data pipeline, known as a "handler" in Marisco terminology, is designed to clean, standardize, and encode [HELCOM data](https://helcom.fi/about-us) into `NetCDF` format. The handler processes raw HELCOM data, applying various transformations and lookups to align it with `MARIS` data standards.

Key functions of this handler:

- **Cleans** and **normalizes** raw HELCOM data
- **Applies standardized nomenclature** and units
- **Encodes the processed data** into `NetCDF` format compatible with MARIS requirements

This handler is a crucial component in the Marisco data processing workflow, ensuring HELCOM data is properly integrated into the MARIS database.


Note: *Additionally, an optional encoder (pipeline) is provided below to process data into a `.csv` format compatible with the MARIS master database. This feature is maintained for legacy purposes, as data ingestion was previously performed using OpenRefine.*

:::{.callout-tip}

For new MARIS users, please refer to [Understanding MARIS Data Formats (NetCDF and Open Refine)](https://github.com/franckalbinet/marisco/blob/main/nbs/metadata/field-definition.ipynb) for detailed information.

:::

The present notebook pretends to be an instance of [Literate Programming](https://www.wikiwand.com/en/articles/Literate_programming) in the sense that it is a narrative that includes code snippets that are interspersed with explanations. When a function or a class needs to be exported in a dedicated python module (in our case `marisco/handlers/helcom.py`) the code snippet is added to the module using `#| exports` as provided by the wonderful [nbdev](https://nbdev.readthedocs.io/en/latest/) library.

In [2]:
#| hide
%load_ext autoreload
%autoreload 2

In [3]:
#| export
import pandas as pd 
import numpy as np
#from functools import partial 
import fastcore.all as fc 
from pathlib import Path 
#from dataclasses import asdict
from typing import List, Dict, Callable, Tuple, Any 
from collections import OrderedDict, defaultdict
import re
from functools import partial

from marisco.utils import (
    Remapper, 
    ddmm_to_dd,
    Match, 
    get_unique_across_dfs
)

from marisco.callbacks import (
    Callback, 
    Transformer, 
    EncodeTimeCB, 
    AddSampleTypeIdColumnCB,
    AddNuclideIdColumnCB, 
    LowerStripNameCB, 
    SanitizeLonLatCB, 
    CompareDfsAndTfmCB, 
    RemapCB
)

from marisco.metadata import (
    GlobAttrsFeeder, 
    BboxCB, 
    DepthRangeCB, 
    TimeRangeCB, 
    ZoteroCB, 
    KeyValuePairCB
)

from marisco.configs import (
    nuc_lut_path, 
    nc_tpl_path, 
    cfg, 
    species_lut_path, 
    sediments_lut_path, 
    bodyparts_lut_path, 
    detection_limit_lut_path, 
    filtered_lut_path, 
    get_lut, 
    unit_lut_path,
    prepmet_lut_path,
    sampmet_lut_path,
    counmet_lut_path, 
    lab_lut_path,
    NC_VARS
)

from marisco.encoders import (
    NetCDFEncoder, 
)

from marisco.decoders import (
    nc_to_dfs,
    get_netcdf_properties, 
    get_netcdf_group_properties,
    get_netcdf_variable_properties
)
import warnings
warnings.filterwarnings('ignore')

In [4]:
#| hide
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)  # Show full column width

## Configuration & file paths

- **fname_in**: path to the folder containing the HELCOM data in CSV format. The path can be defined as a relative path. 

- **fname_out_nc**: path and filename for the NetCDF output.The path can be defined as a relative path. 

- **Zotero key**: used to retrieve attributes related to the dataset from [Zotero](https://www.zotero.org/). The MARIS datasets include a [library](https://maris.iaea.org/datasets) available on [Zotero](https://www.zotero.org/groups/2432820/maris/library). 

- **ref_id**: refers to the location in Archive of the Zotero library.


:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**:  Review NetCDF file name format, see (https://trello.com/c/RlB7mM8N#comment-6747489a3ef094e3520a4272)

:::

In [5]:
#| exports
fname_in = '../../_data/accdb/mors/csv'
fname_out_nc = '../../_data/output/100-HELCOM-MORS-2024.nc'
zotero_key ='26VMZZ2Q' # HELCOM MORS zotero key
ref_id = 100 # HELCOM MORS reference id as defined by MARIS

## Load data

[Helcom MORS (Monitoring of Radioactive Substances in the Baltic Sea) data](https://helcom.fi/about-us) is provided as a Microsoft Access database. 
[`Mdbtools`](https://github.com/mdbtools/mdbtools) can be used to convert the tables of the Microsoft Access database to `.csv` files on Unix-like OS.
Metadata for the HELCOM MORS dataset is available [here](https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24).

**Example steps**:


1. [Download data](https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24)

2. Install mdbtools via VScode Terminal: 

    ```
    sudo apt-get -y install mdbtools
    ```

3. Install unzip via VScode Terminal:

    ```
    sudo apt-get -y install unzip
    ```

4. In `VS Code` terminal (for instance), navigate to the marisco data folder:

    ```
    cd /home/marisco/downloads/marisco/_data/accdb/mors_19840101_20211231
    ```

5. Unzip `MORS_ENVIRONMENT.zip`:

    ```
    unzip MORS_ENVIRONMENT.zip 
    ```

6. Run `preprocess.sh` to generate the required data files:

    ```
    ./preprocess.sh MORS_ENVIRONMENT.zip
    ```

7. Content of `preprocess.sh` script:

    ```
    #!/bin/bash

    # Example of use: ./preprocess.sh MORS_ENVIRONMENT.zip
    unzip $1
    dbname=$(ls *.accdb)
    mkdir csv
    for table in $(mdb-tables -1 "$dbname"); do
        echo "Export table $table"
        mdb-export "$dbname" "$table" > "csv/$table.csv"
    done
    ```

Once converted to `.csv` files, the data is ready to be loaded into a dictionary of dataframes.
    

In [6]:
#| exports
default_smp_types = {  
    'BIO': 'BIOTA', 
    'SEA': 'SEAWATER', 
    'SED': 'SEDIMENT'
}

In [7]:
#| exports
def load_data(src_dir: str|Path, 
              smp_types: dict = default_smp_types 
             ) -> Dict[str, pd.DataFrame]: 
    "Load HELCOM data and return the data in a dictionary of dataframes with the dictionary key as the sample type."
    src_path = Path(src_dir)
    
    def load_and_merge(file_prefix: str) -> pd.DataFrame:
        try:
            df_meas = pd.read_csv(src_path / f'{file_prefix}02.csv')
            df_smp = pd.read_csv(src_path / f'{file_prefix}01.csv')
            return pd.merge(df_meas, df_smp, on='KEY', how='left')
        except FileNotFoundError as e:
            print(f"Error loading files for {file_prefix}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if files are not found
    
    return {smp_type: load_and_merge(file_prefix) for file_prefix, smp_type in smp_types.items()}  

`dfs` is a dictionary of dataframes created from the Helcom dataset located at the path `fname_in`. The data to be included in each dataframe is sorted by sample type. Each dictionary is defined with a key equal to the sample type. 

In [8]:
#| eval: false
dfs = load_data(fname_in)
print('keys/sample types: ', dfs.keys())
for key in dfs.keys():
    print(f'{key} columns: ', dfs[key].columns)

keys/sample types:  dict_keys(['BIOTA', 'SEAWATER', 'SEDIMENT'])
BIOTA columns:  Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'BASIS',
       'ERROR%', 'NUMBER', 'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY',
       'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY', 'STATION',
       'LATITUDE ddmmmm', 'LATITUDE dddddd', 'LONGITUDE ddmmmm',
       'LONGITUDE dddddd', 'SDEPTH', 'RUBIN', 'BIOTATYPE', 'TISSUE', 'NO',
       'LENGTH', 'WEIGHT', 'DW%', 'LOI%', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN',
       'DATE_OF_ENTRY_y'],
      dtype='object')
SEAWATER columns:  Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/m³', 'VALUE_Bq/m³', 'ERROR%_m³',
       'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR',
       'MONTH', 'DAY', 'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'TDEPTH', 'SDEPTH', 'SALIN',
       'TTEMP', 'FILT', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'DATE_OF_ENTRY_y'],
      dtype='object')
SEDIMEN

In [9]:
dfs['SEDIMENT'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y'],
      dtype='object')

## Normalize nuclide names

### Lower & strip nuclide names

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Some nuclide names contain one or multiple trailing spaces.

:::

This is demonstrated below for the `NUCLIDE` column:

In [10]:
#| eval: false
df = get_unique_across_dfs(load_data(fname_in), 'NUCLIDE', as_df=True, include_nchars=True)
df['stripped_chars'] = df['value'].str.strip().str.replace(' ', '').str.len()
print(df[df['n_chars'] != df['stripped_chars']])

    index      value  n_chars  stripped_chars
5       5    SR90           7               4
7       7     CS137         6               5
12     12   K40             8               3
13     13   AM241           8               5
15     15   CO60            8               4
21     21  CS137            9               5
25     25     SR90          6               4
42     42   SR90            8               4
43     43   CS137           8               5
49     49    TC99           7               4
50     50   CS134           8               5
91     91      SR90         5               4
93     93   PU238           8               5


To fix this issue, we use the `LowerStripNameCB` callback. For each dataframe in the dictionary of dataframes, it corrects the nuclide name by converting it lowercase, striping any leading or trailing whitespace(s).

In [11]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripNameCB(col_src='NUCLIDE')])

for key in tfm().keys():
    print(f'{key} nuclides: ')
    print(tfm()[key]['NUCLIDE'].unique())

BIOTA nuclides: 
['cs134' 'k40' 'co60' 'cs137' 'sr90' 'ag108m' 'mn54' 'co58' 'ag110m'
 'zn65' 'sb125' 'pu239240' 'ru106' 'be7' 'ce144' 'pb210' 'po210' 'sb124'
 'sr89' 'zr95' 'te129m' 'ru103' 'nb95' 'ce141' 'la140' 'i131' 'ba140'
 'pu238' 'u235' 'bi214' 'pb214' 'pb212' 'tl208' 'ac228' 'ra223' 'eu155'
 'ra226' 'gd153' 'sn113' 'fe59' 'tc99' 'co57' 'sn117m' 'eu152' 'sc46'
 'rb86' 'ra224' 'th232' 'cs134137' 'am241' 'ra228' 'th228' 'k-40' 'cs138'
 'cs139' 'cs140' 'cs141' 'cs142' 'cs143' 'cs144' 'cs145' 'cs146']
SEAWATER nuclides: 
['cs137' 'sr90' 'h3' 'cs134' 'pu238' 'pu239240' 'am241' 'cm242' 'cm244'
 'tc99' 'k40' 'ru103' 'sr89' 'sb125' 'nb95' 'ru106' 'zr95' 'ag110m'
 'cm243244' 'ba140' 'ce144' 'u234' 'u238' 'co60' 'pu239' 'pb210' 'po210'
 'np237' 'pu240' 'mn54']
SEDIMENT nuclides: 
['ra226' 'cs137' 'ra228' 'k40' 'sr90' 'cs134137' 'cs134' 'pu239240'
 'pu238' 'co60' 'ru103' 'ru106' 'sb125' 'ag110m' 'ce144' 'am241' 'be7'
 'th228' 'pb210' 'co58' 'mn54' 'zr95' 'ba140' 'po210' 'ra224' 'nb95'
 'p

### Remap nuclide names to MARIS data formats

Below, we map nuclide names used by HELCOM to the MARIS standard nuclide names. 

Remapping data provider nomenclatures to MARIS standards is a recurrent operation and is done in a semi-automated manner according to the following pattern:

1. **Inspect** data provider nomenclature:
2. **Match** automatically against MARIS nomenclature (using a fuzzy matching algorithm); 
3. **Fix** potential mismatches; 
4. **Apply** the lookup table to the dataframe.

We will refer to this process as **IMFA** (**I**nspect, **M**atch, **F**ix, **A**pply).

The `get_unique_across_dfs` function is a utility in MARISCO that retrieves unique values from a specified column across all DataFrames. 
Note that there is one DataFrame for each sample type, such as biota, sediment, etc.

In [12]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripNameCB(col_src='NUCLIDE')])

dfs_output = tfm()

# Transpose to display the dataframe horizontally
get_unique_across_dfs(dfs_output, col_name='NUCLIDE', as_df=True).T

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
index,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
value,ru103,cs146,co60,fe59,u235,sr90,po210,pb210,am241,mn54,...,u234,ba140,sr89,cs143,pu241,cs145,cm242,pu239,pu238,cs139


Let's now create an instance of a [fuzzy matching algorithm](https://www.wikiwand.com/en/articles/Approximate_string_matching) `Remapper`. This instance will match the nuclide names of the HELCOM dataset to the MARIS standard nuclide names.

In [13]:
#| eval: false
remapper = Remapper(provider_lut_df=get_unique_across_dfs(dfs_output, col_name='NUCLIDE', as_df=True),
                    maris_lut_fn=nuc_lut_path,
                    maris_col_id='nuclide_id',
                    maris_col_name='nc_name',
                    provider_col_to_match='value',
                    provider_col_key='value',
                    fname_cache='nuclides_helcom.pkl')

Lets try to match HELCOM nuclide names to MARIS standard nuclide names as automatically as possible. The `match_score` column allows to assess the results:

In [14]:
#| eval: false
remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/77 [00:00<?, ?it/s]

Processing: 100%|██████████| 77/77 [00:02<00:00, 31.06it/s]

63 entries matched the criteria, while 14 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
pu239240,pu239,pu239240,3
pu238240,pu240,pu238240,3
cm243244,cm242,cm243244,3
cs134137,cs137,cs134137,3
cs142,ce140,cs142,2
cs143,ce140,cs143,2
cs145,ce140,cs145,2
cs146,cs136,cs146,1
cs140,ce140,cs140,1


We can now manually inspect the unmatched nuclide names and create a table to correct them to the MARIS standard:

In [15]:
#| exports
fixes_nuclide_names = {
    'cs134137': 'cs134_137_tot',
    'cm243244': 'cm243_244_tot',
    'pu239240': 'pu239_240_tot',
    'pu238240': 'pu238_240_tot',
    'cs143': 'cs137',
    'cs145': 'cs137',
    'cs142': 'cs137',
    'cs141': 'cs137',
    'cs144': 'cs137',
    'k-40': 'k40',
    'cs140': 'cs137',
    'cs146': 'cs137',
    'cs139': 'cs137',
    'cs138': 'cs137'
    }

We now include the table `fixes_nuclide_names`, which applies manual corrections to the nuclide names before the remapping process. 
The `generate_lookup_table` function has an `overwrite` parameter (default is `True`), which, when set to `True`, creates a pickle file cache of the lookup table. We can now test the remapping process:

In [16]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_nuclide_names)
fc.test_eq(len(remapper.select_match(match_score_threshold=1, verbose=True)), 0)

Processing:   0%|          | 0/77 [00:00<?, ?it/s]

Processing: 100%|██████████| 77/77 [00:02<00:00, 33.24it/s]

77 entries matched the criteria, while 0 entries had a match score of 1 or higher.


Test passes! We can now create a callback `RemapNuclideNameCB` to remap the nuclide names. Note that we pass `overwrite=False` to the `Remapper` constructor to now use the cached version.


In [17]:
#| exports
# Create a lookup table for nuclide names
lut_nuclides = lambda df: Remapper(provider_lut_df=df,
                                   maris_lut_fn=nuc_lut_path,
                                   maris_col_id='nuclide_id',
                                   maris_col_name='nc_name',
                                   provider_col_to_match='value',
                                   provider_col_key='value',
                                   fname_cache='nuclides_helcom.pkl').generate_lookup_table(fixes=fixes_nuclide_names, 
                                                                                            as_df=False, overwrite=False)

We now create the callback `RemapNuclideNameCB`, which will remap the nuclide names using the `lut_nuclides` lookup table.

In [18]:
#| exports
class RemapNuclideNameCB(Callback):
    "Remap data provider nuclide names to MARIS nuclide names."
    def __init__(self, 
                 fn_lut: Callable # Function that returns the lookup table dictionary
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        df_uniques = get_unique_across_dfs(tfm.dfs, col_name='NUCLIDE', as_df=True)
        #lut = {k: v.matched_maris_name for k, v in self.fn_lut(df_uniques).items()}    
        lut = {k: v.matched_id for k, v in self.fn_lut(df_uniques).items()}    
        for k in tfm.dfs.keys():
            tfm.dfs[k]['NUCLIDE'] = tfm.dfs[k]['NUCLIDE'].replace(lut)

Let's see it in action, along with the `RemapRdnNameCB` callback:

In [19]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[LowerStripNameCB(col_src='NUCLIDE'),
                            RemapNuclideNameCB(lut_nuclides)
                            ])
dfs_out = tfm()

# For instance
dfs_out['BIOTA'].NUCLIDE.unique()

array([31,  4,  9, 33, 12, 21,  6,  8, 22, 10, 24, 77, 17,  2, 37, 41, 47,
       23, 11, 13, 25, 16, 14, 36, 35, 29, 34, 67, 63, 46, 43, 42, 94, 55,
       50, 40, 53, 87, 92, 86, 15,  7, 93, 85, 91, 90, 51, 59, 76, 72, 54,
       57])

## Standardize Time

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Time/date is provide in the `DATE`, `YEAR`
, `MONTH`, `DAY` columns. Note that the `DATE` contains missing values as indicated below. When missing, we fallback on the `YEAR`, `MONTH`, `DAY` columns. Note also that sometimes `DAY` and `MONTH` contain 0. In this case we systematically set them to 1.

:::

In [20]:
#| eval: false
dfs = load_data(fname_in)
for key in dfs.keys():
    print(f'{key} DATE null values: ', dfs[key]['DATE'].isna().sum())

BIOTA DATE null values:  84
SEAWATER DATE null values:  494
SEDIMENT DATE null values:  741


In [21]:
#| exports
class ParseTimeCB(Callback):
    "Parse and standardize time information in the dataframe."
    def __call__(self, tfm: Transformer):
        for df in tfm.dfs.values():
            self._process_dates(df)

    def _process_dates(self, df: pd.DataFrame) -> None:
        "Process and correct date and time information in the DataFrame."
        df['TIME'] = self._parse_date(df)
        self._handle_missing_dates(df)
        self._fill_missing_time(df)

    def _parse_date(self, df: pd.DataFrame) -> pd.Series:
        "Parse the DATE column if present."
        return pd.to_datetime(df['DATE'], format='%m/%d/%y %H:%M:%S', errors='coerce')

    def _handle_missing_dates(self, df: pd.DataFrame):
        "Handle cases where DAY or MONTH is 0 or missing."
        df.loc[df["DAY"] == 0, "DAY"] = 1
        df.loc[df["MONTH"] == 0, "MONTH"] = 1
        
        missing_day_month = (df["DAY"].isna()) & (df["MONTH"].isna()) & (df["YEAR"].notna())
        df.loc[missing_day_month, ["DAY", "MONTH"]] = 1

    def _fill_missing_time(self, df: pd.DataFrame) -> None:
        "Fill missing time values using YEAR, MONTH, and DAY columns."
        missing_time = df['TIME'].isna()
        df.loc[missing_time, 'TIME'] = pd.to_datetime(
            df.loc[missing_time, ['YEAR', 'MONTH', 'DAY']], 
            format='%Y%m%d', 
            errors='coerce'
        )

Apply the transformer for callbacks `ParseTimeCB`. Then, print the `TIME` data for `seawater`.

In [22]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[ParseTimeCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['SEAWATER'][['TIME']])

                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37347
Number of rows removed         0         0         0 

            TIME
0     2012-05-23
1     2012-05-23
2     2012-06-17
3     2012-05-24
4     2012-05-24
...          ...
20313 2015-06-22
20314 2015-06-23
20315 2015-06-23
20316 2015-06-24
20317 2015-06-24

[20318 rows x 1 columns]


The NetCDF time format requires that time be encoded as the number of milliseconds since a specified origin. In our case, the origin is `1970-01-01`, as indicated in the `cdl.toml` file under the `[vars.defaults.time.attrs]` section.

`EncodeTimeCB` converts the HELCOM `time` format to the MARIS NetCDF `time` format.

In [23]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[ParseTimeCB(),
                            EncodeTimeCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
                            

                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37346
Number of rows removed         0         0         1 



In [24]:
tfm.dfs['SEAWATER'].head()

,KEY,NUCLIDE,METHOD,< VALUE_Bq/m³,VALUE_Bq/m³,ERROR%_m³,DATE_OF_ENTRY_x,COUNTRY,LABORATORY,SEQUENCE,...,LONGITUDE (dddddd),TDEPTH,SDEPTH,SALIN,TTEMP,FILT,MORS_SUBBASIN,HELCOM_SUBBASIN,DATE_OF_ENTRY_y,TIME
0,WKRIL2012003,CS137,NaN,NaN,5.3,32.0,08/20/14 00:00:00,90,KRIL,2012003,...,29.3333,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,1337731200
1,WKRIL2012004,CS137,NaN,NaN,19.9,20.0,08/20/14 00:00:00,90,KRIL,2012004,...,29.3333,NaN,29.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,1337731200
2,WKRIL2012005,CS137,NaN,NaN,25.5,20.0,08/20/14 00:00:00,90,KRIL,2012005,...,23.1500,NaN,0.0,NaN,NaN,NaN,11,3,08/20/14 00:00:00,1339891200
3,WKRIL2012006,CS137,NaN,NaN,17.0,29.0,08/20/14 00:00:00,90,KRIL,2012006,...,27.9833,NaN,0.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,1337817600
4,WKRIL2012007,CS137,NaN,NaN,22.2,18.0,08/20/14 00:00:00,90,KRIL,2012007,...,27.9833,NaN,39.0,NaN,NaN,NaN,11,11,08/20/14 00:00:00,1337817600


In [25]:
tfm.dfs['SEDIMENT'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y',
       'TIME'],
      dtype='object')

## Split Sediment Values

Helcom reports two values for the SEDIMENT sample type: `VALUE_Bq/kg` and `VALUE_Bq/m³`. We need to split this and use a single column `VALUE` for the MARIS standard. We will use the `UNIT` column to identify the reported values. 

Lets take a look at the MARIS unit lookup table:

In [26]:
pd.read_excel(unit_lut_path())

,unit_id,unit,unit_sanitized,ordlist,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,-1,Not applicable,Not applicable,NaN,NaN,NaN,NaN
1,0,NOT AVAILABLE,NOT AVAILABLE,0.0,NaN,NaN,NaN
2,1,Bq/m3,Bq per m3,1.0,Bq/m3,NaN,Bq/m<sup>3</sup>
3,2,Bq/m2,Bq per m2,2.0,NaN,NaN,NaN
4,3,Bq/kg,Bq per kg,3.0,NaN,NaN,NaN
5,4,Bq/kgd,Bq per kgd,4.0,NaN,NaN,NaN
6,5,Bq/kgw,Bq per kgw,5.0,NaN,NaN,NaN
7,6,kg/kg,kg per kg,6.0,NaN,NaN,NaN
8,7,TU,TU,7.0,NaN,NaN,NaN
9,8,DELTA/mill,DELTA per mill,8.0,NaN,NaN,NaN


We will define the columns of interest for the SEDIMENT measurement types:

In [27]:
#| exports
coi_sediment = {
    'kg_type': {
        'VALUE': 'VALUE_Bq/kg',
        'UNCERTAINTY': 'ERROR%_kg',
        'DL': '< VALUE_Bq/kg',
        'UNIT': 3,  # Unit ID for Bq/kg
    },
    'm2_type': {
        'VALUE': 'VALUE_Bq/m²',
        'UNCERTAINTY': 'ERROR%_m²',
        'DL': '< VALUE_Bq/m²',
        'UNIT': 2,  # Unit ID for Bq/m²
    }
}

We define the `SplitSedimentValuesCB` callback to split the sediment entries into separate rows for Bq/kg and Bq/m² values. We use underscore to denote the columns are temporary columns created during the splitting process.


In [28]:
#| exports
class SplitSedimentValuesCB(Callback):
    "Split sediment entries into separate rows for Bq/kg and Bq/m² values"
    def __init__(self, 
                 coi: Dict[str, Dict[str, Any]] # Columns of interest with value, uncertainty, DL columns and units
                ):
        fc.store_attr()
        
    def __call__(self, tfm: Transformer):
        if 'SEDIMENT' not in tfm.dfs:
            return
            
        df = tfm.dfs['SEDIMENT']
        dfs_to_concat = []
        
        # For each measurement type (kg and m2)
        for measure_type, cols in self.coi.items():
            # If any of value/uncertainty/DL exists, keep the row
            has_data = (
                df[cols['VALUE']].notna() | 
                df[cols['UNCERTAINTY']].notna() | 
                df[cols['DL']].notna()
            )
            
            if has_data.any():
                df_measure = df[has_data].copy()
                
                # Copy columns to standardized names
                df_measure['_VALUE'] = df_measure[cols['VALUE']]
                df_measure['_UNCERTAINTY'] = df_measure[cols['UNCERTAINTY']]
                df_measure['_DL'] = df_measure[cols['DL']]
                df_measure['_UNIT'] = cols['UNIT']
                
                dfs_to_concat.append(df_measure)
        
        # Combine all measurement type dataframes
        if dfs_to_concat:
            tfm.dfs['SEDIMENT'] = pd.concat(dfs_to_concat, ignore_index=True)

In [29]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SplitSedimentValuesCB(coi_sediment),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

tfm.dfs['SEDIMENT'].head()

                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     64074
Number of rows removed         0         0         0 



,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y,_VALUE,_UNCERTAINTY,_DL,_UNIT
0,SKRIL2012048,RA226,NaN,NaN,35.0,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00,35.0,26.0,NaN,3
1,SKRIL2012049,RA226,NaN,NaN,36.0,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00,36.0,22.0,NaN,3
2,SKRIL2012050,RA226,NaN,NaN,38.0,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00,38.0,24.0,NaN,3
3,SKRIL2012051,RA226,NaN,NaN,36.0,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00,36.0,25.0,NaN,3
4,SKRIL2012052,RA226,NaN,NaN,30.0,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,NaN,11.0,11.0,NaN,08/20/14 00:00:00,30.0,23.0,NaN,3


## Sanitize value

We allocate each column containing measurement values (named differently across sample types) into a single column `VALUE` and remove NA where needed.

In [30]:
#| exports
coi_val = {'SEAWATER' : {'VALUE': 'VALUE_Bq/m³'},
           'BIOTA':  {'VALUE': 'VALUE_Bq/kg'},
           'SEDIMENT': {'VALUE': '_VALUE'}}


In [31]:
#| exports
class SanitizeValueCB(Callback):
    "Sanitize value/measurement by removing blank entries and populating `value` column."
    def __init__(self, 
                 coi: Dict[str, Dict[str, str]] # Columns of interest. Format: {group_name: {'val': 'column_name'}}
                 ): 
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        for grp, df in tfm.dfs.items():
            value_col = self.coi[grp]['VALUE']
            df.dropna(subset=[value_col], inplace=True)
            df['VALUE'] = df[value_col]

In [32]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SplitSedimentValuesCB(coi_sediment),
                            SanitizeValueCB(coi_val),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')

                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14873     20242     63870
Number of rows removed        20        76       112 



In [33]:
rev_sed=tfm.dfs['SEDIMENT']
rev_sed

,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y,_VALUE,_UNCERTAINTY,_DL,_UNIT,VALUE
0,SKRIL2012048,RA226,NaN,NaN,35.00,26.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,11.0,11.0,NaN,08/20/14 00:00:00,35.000000,26.0,NaN,3,35.000000
1,SKRIL2012049,RA226,NaN,NaN,36.00,22.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,11.0,11.0,NaN,08/20/14 00:00:00,36.000000,22.0,NaN,3,36.000000
2,SKRIL2012050,RA226,NaN,NaN,38.00,24.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,11.0,11.0,NaN,08/20/14 00:00:00,38.000000,24.0,NaN,3,38.000000
3,SKRIL2012051,RA226,NaN,NaN,36.00,25.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,11.0,11.0,NaN,08/20/14 00:00:00,36.000000,25.0,NaN,3,36.000000
4,SKRIL2012052,RA226,NaN,NaN,30.00,23.0,NaN,NaN,NaN,08/20/14 00:00:00,...,NaN,11.0,11.0,NaN,08/20/14 00:00:00,30.000000,23.0,NaN,3,30.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64069,SSTUK2016044,CS137,STUK01,NaN,1.20,12.0,NaN,8.916443,15.0,NaN,...,NaN,3.0,3.0,NaN,NaN,8.916443,15.0,NaN,2,8.916443
64070,SSTUK2016045,CS137,STUK01,NaN,0.79,20.0,NaN,5.992930,23.0,NaN,...,NaN,3.0,3.0,NaN,NaN,5.992930,23.0,NaN,2,5.992930
64071,SSTUK2016050,CS137,STUK01,NaN,512.00,11.0,NaN,2164.945699,14.0,NaN,...,NaN,8.0,8.0,NaN,NaN,2164.945699,14.0,NaN,2,2164.945699
64072,SSTUK2016051,CS137,STUK01,NaN,527.00,6.3,NaN,2523.279045,9.3,NaN,...,NaN,8.0,8.0,NaN,NaN,2523.279045,9.3,NaN,2,2523.279045


## Normalize uncertainty

Function `unc_rel2stan` converts uncertainty from relative uncertainty to standard uncertainty.

In [34]:
#| exports
def unc_rel2stan(
    df: pd.DataFrame, # DataFrame containing measurement and uncertainty columns
    meas_col: str, # Name of the column with measurement values
    unc_col: str # Name of the column with relative uncertainty values (percentages)
) -> pd.Series: # Series with calculated absolute uncertainties
    "Convert relative uncertainty to absolute uncertainty."
    return df.apply(lambda row: row[unc_col] * row[meas_col] / 100, axis=1)

For each sample type in the Helcom dataset, the `UNCERTAINTY` is provided as a relative uncertainty. The column names for both the `VALUE` and the `UNCERTAINTY` vary by sample type. The `coi_units_unc` dictionary defines the column names for the `VALUE` and `UNCERTAINTY` for each sample type.

In [35]:
#| exports
# Columns of interest
coi_units_unc = [('SEAWATER', 'VALUE_Bq/m³', 'ERROR%_m³'),
                 ('BIOTA', 'VALUE_Bq/kg', 'ERROR%'),
                 ('SEDIMENT', '_VALUE', '_UNCERTAINTY')]


NormalizeUncCB callback normalizes the ``UNCERTAINTY`` by converting from relative uncertainty to standard uncertainty. 

In [36]:
#| exports
class NormalizeUncCB(Callback):
    "Convert from relative error % to standard uncertainty."
    def __init__(self, 
                 fn_convert_unc: Callable=unc_rel2stan, # Function converting relative uncertainty to absolute uncertainty
                 coi: List[Tuple[str, str, str]]=coi_units_unc # List of columns of interest
                ):
        fc.store_attr()
    
    def __call__(self, tfm: Transformer):
        for grp, val, unc in self.coi:
            if grp in tfm.dfs:
                df = tfm.dfs[grp]
                df['UNCERTAINTY'] = self.fn_convert_unc(df, val, unc)

Apply the transformer for callback ``NormalizeUncCB``. Then, print the value (i.e. activity per unit ) and standard uncertainty for each sample type.

In [37]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[SplitSedimentValuesCB(coi_sediment),
                            SanitizeValueCB(coi_val),
                            NormalizeUncCB()])
tfm()
print(tfm.dfs['SEAWATER'][['VALUE', 'UNCERTAINTY']][:5])
print(tfm.dfs['BIOTA'][['VALUE', 'UNCERTAINTY']][:5])
print(tfm.dfs['SEDIMENT'][['VALUE', 'UNCERTAINTY']][:5])

   VALUE  UNCERTAINTY
0    5.3        1.696
1   19.9        3.980
2   25.5        5.100
3   17.0        4.930
4   22.2        3.996
        VALUE  UNCERTAINTY
0    0.010140          NaN
1  135.300000     4.830210
2    0.013980          NaN
3    4.338000     0.150962
4    0.009614          NaN
   VALUE  UNCERTAINTY
0   35.0         9.10
1   36.0         7.92
2   38.0         9.12
3   36.0         9.00
4   30.0         6.90


## Remap Biota species

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: RUBIN contains codes that are not found in the HELCOM biota dataset. 
:::

For example, 'CH HI;BA', its not in the HELCOM biota dataset. Lets return the uniue RUBIN of the HELCOM biota dataset:

Other unused RUBIN:

In [38]:
#| eval: false
unique_rubin = dfs['BIOTA']['RUBIN'].unique()
unique_rubin_set = set(unique_rubin)
rubin_lut = list(pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv')['RUBIN'])
unused_rubins = [rune for rune in rubin_lut if rune not in unique_rubin_set]
print("Unused RUBIN names:", unused_rubins)

Unused RUBIN names: ['CH HI;BA', 'SOLE SOL']


We will remap the HELCOM `RUBIN` column to the MARIS `SPECIES` column using the **IMFA** (**I**nspect, **M**atch, **F**ix, **A**pply) pattern. First lets **inspect** the `RUBIN_NAME.csv` file provided by HELCOM, which describes the nomenclature of biota species.



In [39]:
#| eval: false
pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv').head()

,RUBIN_ID,RUBIN,SCIENTIFIC NAME,ENGLISH NAME
0,11,ABRA BRA,ABRAMIS BRAMA,BREAM
1,12,ANGU ANG,ANGUILLA ANGUILLA,EEL
2,13,ARCT ISL,ARCTICA ISLANDICA,ISLAND CYPRINE
3,14,ASTE RUB,ASTERIAS RUBENS,COMMON STARFISH
4,15,CARD EDU,CARDIUM EDULE,COCKLE


In [40]:
#| eval: false
species_lut_path()

Path('/home/marisco/.marisco/lut/dbo_species_2024_11_19.xlsx')

Now we try to **MATCH** the `SCIENTIFIC NAME` column of HELCOM biota dataset to the `species` column of the MARIS species lookup table, again using a `Remapper` object:

In [41]:
#| eval: false
remapper = Remapper(provider_lut_df=pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv'),
                    maris_lut_fn=species_lut_path,
                    maris_col_id='species_id',
                    maris_col_name='species',
                    provider_col_to_match='SCIENTIFIC NAME',
                    provider_col_key='RUBIN',
                    fname_cache='species_helcom.pkl'
                    )

remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/43 [00:00<?, ?it/s]

Processing: 100%|██████████| 43/43 [00:09<00:00,  4.71it/s]

35 entries matched the criteria, while 8 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
STIZ LUC,Sander lucioperca,STIZOSTEDION LUCIOPERCA,10
LAMI SAC,Laminaria japonica,LAMINARIA SACCHARINA,7
CARD EDU,Cardiidae,CARDIUM EDULE,6
CH HI;BA,Macoma balthica,CHARA BALTICA,6
ENCH CIM,Echinodermata,ENCHINODERMATA CIM,5
PSET MAX,Pinctada maxima,PSETTA MAXIMA,5
MACO BAL,Macoma balthica,MACOMA BALTICA,1
STUC PEC,Stuckenia pectinata,STUCKENIA PECTINATE,1


Below, we will correct the entries that were not properly matched by the `Remapper` object:

In [42]:
#| exports
fixes_biota_species = {
    'CHARA BALTICA': 'NOT AVAILABLE', # CHARA BALTICA (RUBIN: CH HI;BA) is not listed in the biota data. 
    'CARDIUM EDULE': 'Cerastoderma edule',
    'LAMINARIA SACCHARINA': 'Saccharina latissima',
    'PSETTA MAXIMA': 'Scophthalmus maximus',
    'STIZOSTEDION LUCIOPERCA': 'Sander luciopercas'}

And give the ``remapper`` another try:

In [43]:
#| eval: false
remapper.generate_lookup_table(fixes=fixes_biota_species)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/43 [00:00<?, ?it/s]

Processing: 100%|██████████| 43/43 [00:06<00:00,  6.87it/s]

39 entries matched the criteria, while 4 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
ENCH CIM,Echinodermata,ENCHINODERMATA CIM,5
MACO BAL,Macoma balthica,MACOMA BALTICA,1
STIZ LUC,Sander lucioperca,STIZOSTEDION LUCIOPERCA,1
STUC PEC,Stuckenia pectinata,STUCKENIA PECTINATE,1


Visual inspection of the remaining unperfectly matched entries seem acceptable to proceed. 

We can now use the generic `RemapCB` callback to perform the remapping of the `RUBIN` column to the `species` column after having defined the lookup table `lut_biota`.

In [44]:
#| exports
lut_biota = lambda: Remapper(provider_lut_df=pd.read_csv(Path(fname_in) / 'RUBIN_NAME.csv'),
                             maris_lut_fn=species_lut_path,
                             maris_col_id='species_id',
                             maris_col_name='species',
                             provider_col_to_match='SCIENTIFIC NAME',
                             provider_col_key='RUBIN',
                             fname_cache='species_helcom.pkl'
                             ).generate_lookup_table(fixes=fixes_biota_species, as_df=False, overwrite=False)

In [45]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA')
    ])
tfm()
tfm.dfs['BIOTA'].columns
# For instance:
print(tfm.dfs['BIOTA']['SPECIES'].unique())

[  99  243   50  139  270  192  191  284   84  269  122   96  287  279
  278  288  286  244  129  275  271  285  283  247  120   59  280  274
  273  290  289  272  277  276   21  282  110  281  245  704 1524]


## Remap Biota tissues
Let's inspect the `TISSUE.csv` file provided by HELCOM describing the tissue nomenclature. Biota tissue is known as `body part` in the maris data set.

In [46]:
#| eval: false
pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv').head()

,TISSUE,TISSUE_DESCRIPTION
0,1,WHOLE FISH
1,2,WHOLE FISH WITHOUT ENTRAILS
2,3,WHOLE FISH WITHOUT HEAD AND ENTRAILS
3,4,FLESH WITH BONES
4,5,FLESH WITHOUT BONES (FILETS)


In [47]:
#| eval: false
remapper = Remapper(provider_lut_df=pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv'),
                    maris_lut_fn=bodyparts_lut_path,
                    maris_col_id='bodypar_id',
                    maris_col_name='bodypar',
                    provider_col_to_match='TISSUE_DESCRIPTION',
                    provider_col_key='TISSUE',
                    fname_cache='tissues_helcom.pkl'
                    )

remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/29 [00:00<?, ?it/s]

Processing: 100%|██████████| 29/29 [00:00<00:00, 81.55it/s]

21 entries matched the criteria, while 8 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
3,Flesh without bones,WHOLE FISH WITHOUT HEAD AND ENTRAILS,20
2,Flesh without bones,WHOLE FISH WITHOUT ENTRAILS,13
8,Soft parts,SKIN/EPIDERMIS,10
5,Flesh without bones,FLESH WITHOUT BONES (FILETS),9
1,Whole animal,WHOLE FISH,5
12,Brain,ENTRAILS,5
15,Stomach and intestine,STOMACH + INTESTINE,3
41,Whole animal,WHOLE ANIMALS,1


We address several entries that were not correctly matched by the Remapper object, as detailed below:"

In [48]:
#| exports
fixes_biota_tissues = {
    'WHOLE FISH WITHOUT HEAD AND ENTRAILS': 'Whole animal eviscerated without head',
    'ENTRAILS': 'Viscera',
    'SKIN/EPIDERMIS': 'Skin'}

In [49]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_biota_tissues)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/29 [00:00<?, ?it/s]

Processing: 100%|██████████| 29/29 [00:00<00:00, 72.67it/s]

24 entries matched the criteria, while 5 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
2,Flesh without bones,WHOLE FISH WITHOUT ENTRAILS,13
5,Flesh without bones,FLESH WITHOUT BONES (FILETS),9
1,Whole animal,WHOLE FISH,5
15,Stomach and intestine,STOMACH + INTESTINE,3
41,Whole animal,WHOLE ANIMALS,1


Visual inspection of the remaining unperfectly matched entries seem acceptable to proceed. 

We can now use the generic `RemapCB` callback to perform the remapping of the `TISSUE` column to the `body_part` column after having defined the lookup table `lut_tissues`.

In [50]:
#| exports
lut_tissues = lambda: Remapper(provider_lut_df=pd.read_csv('../../_data/accdb/mors/csv/TISSUE.csv'),
                               maris_lut_fn=bodyparts_lut_path,
                               maris_col_id='bodypar_id',
                               maris_col_name='bodypar',
                               provider_col_to_match='TISSUE_DESCRIPTION',
                               provider_col_key='TISSUE',
                               fname_cache='tissues_helcom.pkl'
                               ).generate_lookup_table(fixes=fixes_biota_tissues, as_df=False, overwrite=False)

In [51]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA'),
    RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='TISSUE', dest_grps='BIOTA'),
    ])

print(tfm()['BIOTA'][['TISSUE', 'BODY_PART']][:5])


   TISSUE  BODY_PART
0       5         52
1       5         52
2       5         52
3       5         52
4       5         52


## Remap biogroup

:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 

1) Is this needed in NETCDF? Can enum include the species and biogroup LUT?

2) Include the `lut_biogroup_from_biota` callback in utils.ipynb. 

:::


`lut_biogroup_from_biota` reads the file at `species_lut_path()` and from the contents of this file creates a dictionary linking `species_id` to `biogroup_id`.

In [52]:
#| exports
lut_biogroup_from_biota = lambda: get_lut(src_dir=species_lut_path().parent, fname=species_lut_path().name, 
                               key='species_id', value='biogroup_id')

In [53]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA'),
    RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='TISSUE', dest_grps='BIOTA'),
    RemapCB(fn_lut=lut_biogroup_from_biota, col_remap='BIO_GROUP', col_src='SPECIES', dest_grps='BIOTA')
    ])

print(tfm()['BIOTA']['BIO_GROUP'].unique())


[ 4  2 14 11  8  3]


## Remap Sediment types

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The `SEDI` values `56` and `73` are not found in the `SEDIMENT_TYPE.csv` lookup table provided. Note also there are many `nan` values in the `SEDIMENT_TYPE.csv` file.

We reassign them to `-99` for now but should be clarified/fixed. This is demonstrated below.

:::

In [54]:
#| eval: false
df_sed_lut = pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv')
dfs = load_data(fname_in)

sediment_sedi = set(dfs['SEDIMENT'].SEDI.unique())
lookup_sedi = set(df_sed_lut['SEDI'])
missing = sediment_sedi - lookup_sedi
print(f"Missing SEDI values: {missing if missing else 'None'}")

Missing SEDI values: {56.0, nan, 73.0}


:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 
 ``SedRepName`` is used by OpenRefine. ``SedRepName`` is not included in the NetCDF encoding. Description of the `SedRepName` from [MARIS Data Formats
](https://github.com/franckalbinet/marisco/tree/main/install_configure_guide); 'Name of the sediment as reported by the data provider. The sediment name should be stored exactly as provided, without any modifications'. 

This information will be lost with the latest workflow (creating netcdf and decoding to csv) if we do not include strings in the netcdf encoding. 
:::

Once again, we employ the **IMFA** (Inspect, Match, Fix, Apply) pattern to remap the HELCOM sediment types. Let's inspect the `SEDIMENT_TYPE.csv` file provided by HELCOM describing the sediment type nomenclature:

In [55]:
#| eval: false
pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv').head()

,SEDI,SEDIMENT TYPE,RECOMMENDED TO BE USED
0,-99,NO DATA,NaN
1,30,SILT AND GRAVEL,YES
2,0,GRAVEL,YES
3,1,SAND,YES
4,2,FINE SAND,NO


Let's try to match as many as possible:

In [56]:
#| eval: false
remapper = Remapper(provider_lut_df=pd.read_csv(Path(fname_in)/'SEDIMENT_TYPE.csv'),
                    maris_lut_fn=sediments_lut_path,
                    maris_col_id='sedtype_id',
                    maris_col_name='sedtype',
                    provider_col_to_match='SEDIMENT TYPE',
                    provider_col_key='SEDI',
                    fname_cache='sediments_helcom.pkl'
                    )

remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/47 [00:00<?, ?it/s]

Processing: 100%|██████████| 47/47 [00:00<00:00, 71.37it/s]

44 entries matched the criteria, while 3 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
-99,Soft,NO DATA,5
50,Mud and gravel,MUD AND GARVEL,2
46,Glacial clay,CLACIAL CLAY,1


We address the remaining unmatched values by adding fixes_sediments:

In [57]:
#| exports
fixes_sediments = {
    'NO DATA': '(Not available)'
}

In [58]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_sediments)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/47 [00:00<?, ?it/s]

Processing: 100%|██████████| 47/47 [00:00<00:00, 88.59it/s]

45 entries matched the criteria, while 2 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
50,Mud and gravel,MUD AND GARVEL,2
46,Glacial clay,CLACIAL CLAY,1


A visual inspection of the remaining values shows that they are acceptable to proceed.


In [59]:
#| exports
class RemapSedimentCB(Callback):
    "Update sediment id based on MARIS species LUT (dbo_sedtype.xlsx)."
    
    def __init__(self, 
                 fn_lut: Callable,  # Function that returns the lookup table dictionary
                 sed_grp_name: str = 'SEDIMENT',  # The name of the sediment group
                 replace_lut: dict = None  # Dictionary for replacing SEDI values
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        "Remap sediment types in the DataFrame using the lookup table and handle specific replacements."
        lut = self.fn_lut()
        
        # Fix inconsistent SEDI values
        tfm.dfs[self.sed_grp_name] = self._fix_inconsistent_sedi(tfm.dfs[self.sed_grp_name], self.replace_lut)
        
        # Get unique SEDI values
        unique_sedi = tfm.dfs[self.sed_grp_name]['SEDI'].unique()
        
        # Get sediment types for unique SEDI values
        sediment_mapping = self._get_sediment_types(unique_sedi, lut)
        
        # Replace SEDI values in the DataFrame using the mapping
        tfm.dfs[self.sed_grp_name]['SED_TYPE'] = tfm.dfs[self.sed_grp_name]['SEDI'].map(sediment_mapping)

    def _fix_inconsistent_sedi(self, df: pd.DataFrame, replace_lut: dict) -> pd.DataFrame:
        "Temporary fix for inconsistent SEDI values. Data provider to confirm and clarify."
        df['SEDI'] = df['SEDI'].replace(replace_lut)
        return df

    def _get_sediment_types(self, unique_sedi: np.ndarray, lut: dict) -> dict:
        "Get sediment types for unique SEDI values and return a mapping dictionary."
        sediment_mapping = {}
        
        for sedi_value in unique_sedi:
            match = lut.get(sedi_value, Match(0, None, None, None))
            if match.matched_id == 0:
                self._print_unmatched_sedi(sedi_value)
            sediment_mapping[sedi_value] = match.matched_id
        
        return sediment_mapping

    def _print_unmatched_sedi(self, 
                              sedi_value: int,  # The `SEDI` value from the DataFrame
                             ) -> None:
        "Print the SEDI value if the matched_id is 0 (i.e. Not available)."
        print(f"Unmatched SEDI: {sedi_value}")


In [60]:
#| exports
lut_sediments = lambda: Remapper(provider_lut_df=pd.read_csv(Path(fname_in) / 'SEDIMENT_TYPE.csv'),
                                 maris_lut_fn=sediments_lut_path,
                                 maris_col_id='sedtype_id',
                                 maris_col_name='sedtype',
                                 provider_col_to_match='SEDIMENT TYPE',
                                 provider_col_key='SEDI',
                                 fname_cache='sediments_helcom.pkl'
                                 ).generate_lookup_table(fixes=fixes_sediments, as_df=False, overwrite=False)

Reassign the `SEDI` values of `56`, `73`, and `nan` to `-99`:

In [61]:
#| exports
sed_replace_lut = {
    56: -99,
    73: -99,
    np.nan: -99
}

Utilize the RemapSedimentCB callback to remap the SEDI values in the HELCOM dataset to the corresponding MARIS standard sediment type, referred to as SED_TYPE. After the remapping process, display the SEDI and SED_TYPE columns from the SEDIMENT DataFrame.

In [62]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[RemapSedimentCB(fn_lut=lut_sediments, replace_lut=sed_replace_lut)])

tfm()

tfm.dfs['SEDIMENT']['SED_TYPE'].unique()

Unmatched SEDI: -99.0


array([ 0,  2, 58, 30, 59, 55, 56, 36, 29, 47,  4, 54, 33,  6, 44, 42, 48,
       61, 57, 28, 49, 32, 45, 39, 46, 38, 31, 60, 62, 26, 53, 52,  1, 51,
       37, 34, 50,  7, 10, 41, 43, 35])

## Remap units

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: The handling of unit types varies between `biota` and `sediment` sample types. For consistency and ease of use, it would be beneficial to have dedicated unit columns for all sample types.

:::

Given the inconsistent handling of units across sample types, we need to define custom mapping rules for standardizing the units. The units available in MARIS are:

In [63]:
#| eval: false
pd.read_excel(unit_lut_path())[['unit_id', 'unit', 'unit_sanitized']]

,unit_id,unit,unit_sanitized
0,-1,Not applicable,Not applicable
1,0,NOT AVAILABLE,NOT AVAILABLE
2,1,Bq/m3,Bq per m3
3,2,Bq/m2,Bq per m2
4,3,Bq/kg,Bq per kg
5,4,Bq/kgd,Bq per kgd
6,5,Bq/kgw,Bq per kgw
7,6,kg/kg,kg per kg
8,7,TU,TU
9,8,DELTA/mill,DELTA per mill


We define unit renaming rules for HELCOM in an **ad hoc** way:

In [64]:
#| exports
lut_units = {
    'SEAWATER': 1,  # 'Bq/m3'
    'SEDIMENT': 4,  # 'Bq/kgd' for sediment
    'BIOTA': {
        'D': 4,  # 'Bq/kgd'
        'W': 5,  # 'Bq/kgw'
        'F': 5   # 'Bq/kgw' (assumed to be 'Fresh', so set to wet)
    }
}

In [65]:
#| exports
class RemapUnitCB(Callback):
    "Set the `unit` id column in the DataFrames based on a lookup table."
    def __init__(self, 
                 lut_units: dict=lut_units # Dictionary containing renaming rules for different unit categories
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        for grp in tfm.dfs.keys():
            if grp in ['SEAWATER', 'SEDIMENT']:
                tfm.dfs[grp]['UNIT'] = self.lut_units[grp]
            else:
                tfm.dfs[grp]['UNIT'] = tfm.dfs[grp]['BASIS'].apply(lambda x: lut_units[grp].get(x, 0))

Apply the transformer for callback `RemapUnitCB()`. Then, print the unique `UNIT` for the `SEAWATER` dataframe.

In [66]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[RemapUnitCB()])

for grp in ['BIOTA', 'SEDIMENT', 'SEAWATER']:
    print(f"{grp}: {tfm()[grp]['UNIT'].unique()}")

BIOTA: [5 0 4]
SEDIMENT: [4]
SEAWATER: [1]


## Remap detection limit
Detection limits are encoded as follows in MARIS:

In [67]:
#| eval: false
pd.read_excel(detection_limit_lut_path())

,id,name,name_sanitized
0,-1,Not applicable,Not applicable
1,0,Not Available,Not available
2,1,=,Detected value
3,2,<,Detection limit
4,3,ND,Not detected
5,4,DE,Derived


In [68]:
#| exports
lut_dl = lambda: pd.read_excel(detection_limit_lut_path(), usecols=['name','id']).set_index('name').to_dict()['id']

Based on columns of interest for each sample type:

In [69]:
#| exports
coi_dl = {'SEAWATER' : {'VALUE' : 'VALUE_Bq/m³',
                       'UNCERTAINTY' : 'ERROR%_m³',
                       'DL' : '< VALUE_Bq/m³'},
          'BIOTA':  {'VALUE' : 'VALUE_Bq/kg',
                     'UNCERTAINTY' : 'ERROR%',
                     'DL' : '< VALUE_Bq/kg'},
          'SEDIMENT': {
              'VALUE' : 'VALUE_Bq/kg',
              'UNCERTAINTY' : 'ERROR%_kg',
              'DL' : '< VALUE_Bq/kg'}}


We follow the following business logic to encode the detection limit:

`RemapDetectionLimitCB` creates a `detection_limit` column with values determined as follows:
1. Perform a lookup with the appropriate columns value type (or DL) columns (`< VALUE_Bq/m³` or `< VALUE_Bq/kg`) against the table returned from the function `get_detectionlimit_lut`.
2. If `< VALUE_Bq/m³` or `< VALUE_Bq/kg` is NaN but both activity values (`VALUE_Bq/m³` or `VALUE_Bq/kg`) and standard uncertainty (`ERROR%_m³`, `ERROR%`, or `ERROR%_kg`) are provided, then assign the ID of `1` (i.e. "Detected value").
3. For other NaN values in the `detection_limit` column, set them to `0` (i.e. `Not Available`).

In [70]:
# | exports
class RemapDetectionLimitCB(Callback):
    "Remap value type to MARIS format."
    
    def __init__(self, 
                 coi: dict,  # Configuration options for column names
                 fn_lut: Callable  # Function that returns a lookup table
                ):
        fc.store_attr()

    def __call__(self, tfm: Transformer):
        "Remap detection limits in the DataFrames using the lookup table."
        lut = self.fn_lut()
        
        for grp in tfm.dfs:
            df = tfm.dfs[grp]
            self._update_detection_limit(df, grp, lut)

    def _update_detection_limit(self, 
                                df: pd.DataFrame,  # The DataFrame to modify
                                grp: str,  # The group name to get the column configuration
                                lut: dict  # The lookup table dictionary
                               ) -> None:
        "Update detection limit column in the DataFrame based on lookup table and rules."
        
        # Check if the group exists in coi_dl
        if grp not in coi_dl:
            raise ValueError(f"Group '{grp}' not found in coi_dl configuration.")
        
        # Access column names from coi_dl
        value_col = coi_dl[grp]['VALUE']
        uncertainty_col = coi_dl[grp]['UNCERTAINTY']
        detection_col = coi_dl[grp]['DL']

        # Initialize detection limit column
        df['DL'] = df[detection_col]
        
        # Set detection limits based on conditions
        self._set_detection_limits(df, value_col, uncertainty_col, lut)

    def _set_detection_limits(self, df: pd.DataFrame, value_col: str, uncertainty_col: str, lut: dict) -> None:
        "Set detection limits based on value and uncertainty columns."
        
        # Condition for setting '='
        # 'DL' defaults to equal (i.e. '=') if there is a value and uncertainty and 'DL' value is not 
        # in the lookup table.
        
        condition_eq =(df[value_col].notna() & 
                       df[uncertainty_col].notna() & 
                       ~df['DL'].isin(lut.keys())
        )
        
        df.loc[condition_eq, 'DL'] = '='

        # Set 'Not Available' for unmatched detection limits
        df.loc[~df['DL'].isin(lut.keys()), 'DL'] = 'Not Available'
        
        # Perform lookup to map detection limits
        df['DL'] = df['DL'].map(lut)

In [71]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            SplitSedimentValuesCB(coi_sediment),
                            NormalizeUncCB(),
                            SanitizeValueCB(coi_val),                       
                            RemapUnitCB(),
                            RemapDetectionLimitCB(coi_dl, lut_dl)])


for grp in ['BIOTA', 'SEDIMENT', 'SEAWATER']:
    print(f"{grp}: {tfm()[grp]['DL'].unique()}")

BIOTA: [2 1 0]
SEDIMENT: [1 2 0]
SEAWATER: [1 2 0]


## Remap filtering status

HELCOM filtered status is encoded as follows in the `FILT` column:

In [72]:
#| eval: false
dfs = load_data(fname_in)
get_unique_across_dfs(dfs, col_name='FILT', as_df=True).head(5)

,index,value
0,0,NaN
1,1,F
2,2,N
3,3,n


MARIS uses a different encoding for filtered status:

In [73]:
#| eval: false
pd.read_excel(filtered_lut_path())

,id,name
0,-1,Not applicable
1,0,Not available
2,1,Yes
3,2,No


For only four categories to remap, the `Remapper` is an overkill. We can use a simple dictionary to map the values:

In [74]:
#| exports
lut_filtered = {
    'N': 2, # No
    'n': 2, # No
    'F': 1 # Yes
}

`RemapFiltCB` converts the HELCOM `FILT` format to the MARIS `FILT` format.

In [75]:
#| exports
class RemapFiltCB(Callback):
    "Lookup FILT value in dataframe using the lookup table."
    def __init__(self,
                 lut_filtered: dict=lut_filtered, # Dictionary mapping FILT codes to their corresponding names
                ):
        fc.store_attr()

    def __call__(self, tfm):
        for df in tfm.dfs.values():
            if 'FILT' in df.columns:
                df['FILT'] = df['FILT'].map(lambda x: self.lut_filtered.get(x, 0))

For instance:

In [76]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[RemapFiltCB(lut_filtered)])

print(tfm()['SEAWATER']['FILT'].unique())


[0 2 1]


## Add Laboratory ID (REVIEW)

:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**:  Review the inclusion of LAB in the NetCDF output, note with minor updates to dbo_lab.xlsx it would offer a way to obtain a `SMP_ID`

This section could be simplified by including all Helcom 'LABORATORY' names in the MARIS standard laboratory names lookup table (dbo_lab.xlsx). For example STUK, KRIL, RISO, etc. are absent from the MARIS standard laboratory names lookup table lab_abb column.

:::

Lets use the utility `get_unique_across_dfs` function to review the unique laboratory IDs in the HELCOM dataset:

In [77]:
#| eval: false
# Transpose to display the dataframe horizontally
get_unique_across_dfs(tfm.dfs, col_name='LABORATORY', as_df=True).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
index,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
value,NaN,SAAS,SSSI,STUK,VTIG,IMGW,LEPA,JORC,CLOR,EBRS,LREB,RISO,LVDC,KRIL,NCRS,SSSM,BFFG,LVEA,DHIG,ERPC


The HELCOM dataset includes a lookup table `LABORATORY_NAME.csv` which captures the laboratory names and codes.

In [78]:
#| eval: false
pd.read_csv(Path(fname_in) / 'LABORATORY_NAME.csv').head()

,LABORATORY,LABORATORY_NAME,START_DATE,END_DATE,COUNTRY
0,BFFG,"BUNDESFORSCHUNGANSTALT FÜR FISCHEREI, GERMANY",01/01/86 00:00:00,12/31/07 00:00:00,6
1,CLOR,"CENTRAL LABORATORY FOR RADIOLOGICAL PROTECTION, POLAND",01/01/84 00:00:00,NaN,67
2,DHIG,"FEDERAL MARITIME AND HYDROGRAPHIC AGENCY, GERMANY",01/01/84 00:00:00,NaN,6
3,EBRS,"RADIATION SAFETY DEPARTMENT ENVIRONMENTAL BOARD, ESTONIA",01/01/10 00:00:00,NaN,91
4,EMHI,"ESTONIAN METEOROLOGICAL AND HYDROLOGICAL INSTITUTE, ESTONIA",NaN,NaN,91


Lets take a look at the MARIS standard laboratory names:

In [79]:
#| eval: false
maris_lab_lut=pd.read_excel(lab_lut_path())
maris_lab_lut.head(4)

,lab_id,lab_abb,lab,addr_1,addr_2,twn_zip,country,tel,e_mail,fax,note,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,-1,Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Not available,Not available,NaN,NaN,NaN,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,IAEA-EL,International Atomic Energy Agency - Environment Laboratory (former Marine Environment Laboratory),NaN,P.O. Box No. 800,MC-98012 Monaco Cedex,Principality of Monaco,NaN,NaN,NaN,NaN,NaN,NaN,"update lab set lab = 'International Atomic Energy Agency - Environment Laboratory (former Marine Environment Laboratory)', country = '' where lab_id = 1"
3,2,INPAS,Institute of Nuclear Physics - Academy of Sciences,NaN,NaN,Tirana,Albania,NaN,NaN,NaN,NaN,NaN,NaN,"update lab set lab = 'Institute of Nuclear Physics - Academy of Sciences', country = '' where lab_id = 2"


::: {.callout-tip}

**FEEDBACK FOR DATA PROVIDER**: 
One entry for the `LABORATORY` column includes a 'NaN', see below.

:::

In [80]:
#| eval: false

def find_nan_entries(dfs, columns=None):
    """
    Returns a dictionary of DataFrames, each containing the complete rows where any of the specified columns have NaN values from the original DataFrames.
    
    Parameters:
        dfs (dict): A dictionary where keys are dataset names and values are pandas DataFrames.
        columns (list, optional): A list of column names to check for NaN values. If None, all columns are checked.
    
    Returns:
        dict: A dictionary where each key is the dataset name and the value is a DataFrame of complete rows that have NaN entries in the specified columns.
    """
    nan_entries = {}
    for key, df in dfs.items():
        # If columns are specified, check these columns for NaN values
        if columns is not None:
            # Find rows with NaN values in the specified columns
            nan_rows = df[columns].isnull().any(axis=1)
        else:
            # Find rows with any NaN values across all columns
            nan_rows = df.isnull().any(axis=1)
        
        # Use the boolean index to select the complete rows from the original DataFrame
        complete_nan_rows = df[nan_rows]
        
        if not complete_nan_rows.empty:
            nan_entries[key] = complete_nan_rows
    return nan_entries

nan_lab_df = find_nan_entries(dfs, columns=['LABORATORY'])

print ('Entries with NaN in the `LABORATORY` column:')
for key, df in nan_lab_df.items():
    print(f"{key}: \n{df}")

Entries with NaN in the `LABORATORY` column:
SEDIMENT: 
                KEY NUCLIDE  METHOD < VALUE_Bq/kg  VALUE_Bq/kg  ERROR%_kg  \
35783  SDHIG2016236   CS137  DHIG03           NaN       8.2952      2.351   

      < VALUE_Bq/m²  VALUE_Bq/m²  ERROR%_m²    DATE_OF_ENTRY_x  ...  LOWSLI  \
35783           NaN   237.500899        NaN  05/13/19 00:00:00  ...     NaN   

      AREA  SEDI OXIC  DW%  LOI%  MORS_SUBBASIN HELCOM_SUBBASIN  SUM_LINK  \
35783  NaN   NaN  NaN  NaN   NaN            NaN             NaN       NaN   

       DATE_OF_ENTRY_y  
35783              NaN  

[1 rows x 35 columns]


:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 
Consider integrating combine_lut_columns into utils.ipynb. I've updated the remapper and match_maris_lut functions to accept either a lut_path or a DataFrame. This code could be further simplified by handling the file opening (e.g., pd.read_excel) directly within the remapper function, thereby always passing a DataFrame to match_maris_lut. Refer to the implementation in utils.ipynb for details.

:::

The HELCOM description of laboratory includes both the laboratory name and country. Lets update the ``maris_lab_lut`` to include the laboratory name and country in the same column. 

In [81]:
#| exports
def combine_lut_columns(lut_path: Callable, combine_cols: List[str] = []):
    if lut_path:
        df_lut = pd.read_excel(lut_path()) 
        if combine_cols:
            # Combine the specified columns into a single column with space as separator
            df_lut['combined'] = df_lut[combine_cols].astype(str).agg(' '.join, axis=1)
            # Create a column name by joining column names with '_'
            combined_col_name = '_'.join(combine_cols)
            df_lut.rename(columns={'combined': combined_col_name}, inplace=True)
        return df_lut

In [82]:
#| eval: false
df_lut=combine_lut_columns(lab_lut_path, ['lab','country'])
df_lut.head(3)

,lab_id,lab_abb,lab,addr_1,addr_2,twn_zip,country,tel,e_mail,fax,note,Unnamed: 11,Unnamed: 12,Unnamed: 13,lab_country
0,-1,Not applicable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan nan
1,0,Not available,Not available,NaN,NaN,NaN,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available Not available
2,1,IAEA-EL,International Atomic Energy Agency - Environment Laboratory (former Marine Environment Laboratory),NaN,P.O. Box No. 800,MC-98012 Monaco Cedex,Principality of Monaco,NaN,NaN,NaN,NaN,NaN,NaN,"update lab set lab = 'International Atomic Energy Agency - Environment Laboratory (former Marine Environment Laboratory)', country = '' where lab_id = 1",International Atomic Energy Agency - Environment Laboratory (former Marine Environment Laboratory) Principality of Monaco


Let's now create an instance of a [fuzzy matching algorithm](https://www.wikiwand.com/en/articles/Approximate_string_matching) `Remapper`. This instance will match the ``LABORATORY`` column of the HELCOM dataset to the MARIS standard laboratory names using both `lab` and `country` fields.

In [83]:
#| eval: false
remapper = Remapper(provider_lut_df=pd.read_csv(Path(fname_in) / 'LABORATORY_NAME.csv'),
                    maris_lut_fn= combine_lut_columns(lut_path=lab_lut_path, combine_cols=['lab','country']),
                    maris_col_id='lab_id',
                    maris_col_name='lab_country',
                    provider_col_to_match='LABORATORY_NAME',
                    provider_col_key='LABORATORY',
                    fname_cache='lab_helcom.pkl')

Lets try to match ``LABORATORY`` names to MARIS standard laboratory names as automatically as possible. The `match_score` column allows to assess the results:

In [84]:
#| eval: false
remapper.generate_lookup_table(as_df=True)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/20 [00:00<?, ?it/s]

Processing: 100%|██████████| 20/20 [00:00<00:00, 64.19it/s]

0 entries matched the criteria, while 20 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
SSSI,Nuclear Research Institute Vietnam,"STATENS STRÅLSKYDDSINSTITUT, SWEDEN",23
KRIL,Polytechnic Institute Romania,"V. G. KHLOPIN RADIUM INSTITUTE, RUSSIA",22
STUK,Radiation and Nuclear Safety Authority Finland,"SÄTEILYTURVAKESKUS, RADIATION AND NUCLEAR SAFETY AUTHORITY, FINLAND",21
SAAS,National Board of Nuclear Safety and Radiation Protection Germany,"NATIONAL BOARD FOR ATOMIC SAFETY AND RADIATION PROTECTION, GERMANY",10
RISO,Risø National Laboratory - The Radiation Research Department Denmark,"RISÖ NATIONAL LABORATORY, RADIATION RESEARCH DEPARTMENT, DENMARK",8
LEPA,Environmental Protection Agency Ireland,"ENVIRONMENTAL PROTECTION AGENCY, LITHUANIA",7
NCRS,The Swedish University of Agricultural Sciences Sweden,"SWEDISH UNIVERSITY OF AGRICULTURAL SCIENCES, SWEDEN",5
CLOR,Central Laboratory for Radiological Protection Poland,"CENTRAL LABORATORY FOR RADIOLOGICAL PROTECTION, POLAND",4
VTIG,JOHAN HEINRICH VON THÜNEN-INSTITUTE Germany,"JOHANN HEINRICH VON THÜNEN-INSTITUTE, GERMANY",2


Although the match score is 1 or greater for all entries, many are still matched appropriately. Let's manually correct any unmatched values. Here, we are manually aligning the data providers' laboratory names with those used by the MARIS LUT.

In [85]:
#| exports
fixes_lab_names = {
    'STATENS STRÅLSKYDDSINSTITUT, SWEDEN': 'Swedish Radiation Safety Authority Sweden',
    'V. G. KHLOPIN RADIUM INSTITUTE, RUSSIA': 'V.G. Khlopin Radium Institute - Lab. of Environmental Radioactive Contamination Monitoring Russian Federation',
    'ENVIRONMENTAL PROTECTION AGENCY, LITHUANIA': 'Lithuanian Environmental Protection Agency Lithuania',
    }

Now, lets apply the manual corrections, `fixes_lab_names` and try again. 

In [86]:
#| eval: false
remapper.generate_lookup_table(as_df=True, fixes=fixes_lab_names)
remapper.select_match(match_score_threshold=1, verbose=True)

Processing:   0%|          | 0/20 [00:00<?, ?it/s]

Processing: 100%|██████████| 20/20 [00:00<00:00, 54.38it/s]

3 entries matched the criteria, while 17 entries had a match score of 1 or higher.


,matched_maris_name,source_name,match_score
source_key,,,
STUK,Radiation and Nuclear Safety Authority Finland,"SÄTEILYTURVAKESKUS, RADIATION AND NUCLEAR SAFETY AUTHORITY, FINLAND",21
SAAS,National Board of Nuclear Safety and Radiation Protection Germany,"NATIONAL BOARD FOR ATOMIC SAFETY AND RADIATION PROTECTION, GERMANY",10
RISO,Risø National Laboratory - The Radiation Research Department Denmark,"RISÖ NATIONAL LABORATORY, RADIATION RESEARCH DEPARTMENT, DENMARK",8
NCRS,The Swedish University of Agricultural Sciences Sweden,"SWEDISH UNIVERSITY OF AGRICULTURAL SCIENCES, SWEDEN",5
CLOR,Central Laboratory for Radiological Protection Poland,"CENTRAL LABORATORY FOR RADIOLOGICAL PROTECTION, POLAND",4
VTIG,JOHAN HEINRICH VON THÜNEN-INSTITUTE Germany,"JOHANN HEINRICH VON THÜNEN-INSTITUTE, GERMANY",2
EBRS,"Radiation Safety Department, Environmental Board Estonia","RADIATION SAFETY DEPARTMENT ENVIRONMENTAL BOARD, ESTONIA",2
SSSM,SVERIGE'S STRÅL SÄKERHETS MYNDIGHETEN Sweden,"SVERIGE'S STRÅLSÄKERHETS MYNDIGHETEN, SWEDEN",2
LVEA,Latvian Environment Agency Latvia,"LATVIAN ENVIRONMENT AGENCY, LATVIA",1


We have successfully matched the laboratory names to the MARIS standard laboratory names. We can now create a lookup table for the laboratory names.

In [87]:
#| exports
# Create a lookup table for laboratory names
lut_lab = lambda: Remapper(provider_lut_df=pd.read_csv(Path(fname_in) / 'LABORATORY_NAME.csv'),
                    maris_lut_fn= combine_lut_columns(lut_path=lab_lut_path, combine_cols=['lab','country']),
                    maris_col_id='lab_id',
                    maris_col_name='lab_country',
                    provider_col_to_match='LABORATORY_NAME',
                    provider_col_key='LABORATORY',
                    fname_cache='lab_helcom.pkl').generate_lookup_table(fixes=fixes_lab_names,as_df=False, overwrite=False) 

We now create the callback `RemapLabCB`, which will remap the nuclide names using the `lut_lab` lookup table.

In [88]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
    RemapCB(fn_lut=lut_lab, col_remap='LAB', col_src='LABORATORY', dest_grps=['BIOTA','SEDIMENT','SEAWATER'])
    ])
tfm()
tfm.dfs['BIOTA'].columns
# For instance:
unique_labs = tfm.dfs['BIOTA'][['LABORATORY', 'LAB']].drop_duplicates()
print('Example of unique laboratory names: \n', unique_labs[:20])

Unmatched value: nan
Example of unique laboratory names: 
       LABORATORY  LAB
0           VTIG  342
547         STUK  301
2855        SSSM  340
3095        SSSI  381
3611        SAAS  118
4249        RISO  329
5109        NCRS  238
5828        LVEA  325
5850        LVDC  326
5866        LREB  327
5875        LEPA  324
5916        JORC  323
5940        ERPC  322
5977        EBRS  350
5999        CLOR  188
8103        BFFG  341
14055       IMGW  191


## Add Sample ID (REVIEW)

::: {.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 
Enhancing traceability of NetCDF entries to original samples in the datasource using a standardized `SMP_ID`.

*Context*:

Previously, the NetCDF output did not include a sample laboratory code (or `SMP_ID`), limiting our ability to trace data back to its source. 

*Issue Identified*:
The `KEY` column in the HELCOM dataset, which combines a sample type, a laboratory code, and an integer sequence offers a way trace data back to the HELCOM source. The `KEY` is of type string which is not included in our NetCDF output. To include a way to trace data back to the HELCOM source, we propose to include a `SMP_ID` in the NetCDF output which is of type integer.

*Proposed Solution*:
For the HELCOM dataset, where the `KEY` column includes unique codes like `WDHIG1996246` (comprising sample type, lab code, and sequence), we propose encoding this into a structured `SMP_ID`. This `SMP_ID` will use standardized MARIS Lookup Tables (LUTs) to convert both the sample type and laboratory code into integers.

*Implementation Details*:
- The `SMP_ID` will be formatted such that:
  - The first digit indicates the sample type (e.g., 1 for Seawater).
  - The next three digits represent the laboratory code (e.g., 313 for DHIG as standardized in dbo_lab.xlsx).
  - The remaining digits reflect the integer sequence from the HELCOM KEY.
- Example: `WDHIG1996246` becomes `SMP_ID` `13131996246`.

*Action Required*:
To adopt this approach, a review and update of the laboratory codes in the LUT (dbo_lab.xlsx) are necessary to ensure consistency and accuracy. 

:::

First we wil use ``check_unique_key_int`` to show the non unique integer part of the `KEY` column.

In [96]:
#| eval: false
def check_unique_key_int(tfm):
    """
    Extracts unique 'KEY' values from specified DataFrames, separates them into string and integer components,
    and groups keys by their integer components.

    Parameters:
    tfm (Transformer): The transformer object containing DataFrames.

    Returns:
    dict: A dictionary with the unique keys, their string and integer components, and grouped keys by integer component.
    """
    # Define the groups to extract keys from
    groups = ['SEAWATER', 'BIOTA', 'SEDIMENT']
    
    # Initialize a set to store unique keys
    unique_keys = set()
    
    # Collect unique keys from each DataFrame
    for grp in groups:
        unique_keys.update(tfm.dfs[grp]['KEY'].unique())
    
    # Initialize a dictionary to group keys by their integer components
    int_key_map = {}
    
    for key in unique_keys:
        # Assuming the integer part starts after the first 5 characters
        int_part = int(key[5:]) if key[5:].isdigit() else None  # Remaining part as integer
        
        if int_part is not None:
            if int_part not in int_key_map:
                int_key_map[int_part] = []  # Initialize list for this integer part
            int_key_map[int_part].append(key)  # Append the complete key to the list
    
    return {
        'int_key_map': int_key_map  # Return the mapping of integer parts to complete keys
    }

Below, we will generate a DataFrame where the index (labeled 'INT COMPONENT OF `KEY`') represents the integer portion extracted from the Helcom `KEY`. The 'KEYS' column lists all the `KEY` values that include this integer component. Originally, the plan was to use the integer part of the `KEY` column to create the `SMP_ID`. However, as demonstrated below, the integer part is not unique, which complicates this approach.

In [97]:
#| eval: false
# Create DataFrame from dictionary and set index name and column name
unique_key_df = pd.DataFrame.from_dict(check_unique_key_int(tfm)).rename_axis('INT COMPONENT OF `KEY`')
unique_key_df=unique_key_df.rename(columns={unique_key_df.columns[0]: 'KEYS'})
unique_key_df.head(5)

,KEYS
INT COMPONENT OF `KEY`,
2006012,"[SRISO2006012, BSSSI2006012, SCLOR2006012, WSTUK2006012, WKRIL2006012, BSTUK2006012, BCLOR2006012, BRISO2006012, BBFFG2006012, WIMGW2006012, WLEPA2006012, SKRIL2006012, WRISO2006012, SSTUK2006012, SSSSI2006012]"
2010003,"[BVTIG2010003, BRISO2010003, WSSSI2010003, SLEPA2010003, WLVEA2010003, WEBRS2010003, BSSSM2010003, WKRIL2010003, SLVEA2010003, WRISO2010003, WLEPA2010003, SSTUK2010003, BCLOR2010003, SSSSI2010003, SRISO2010003, BSTUK2010003, WSTUK2010003, WIMGW2010003, SKRIL2010003, SCLOR2010003, BEBRS2010003]"
2012014,"[WKRIL2012014, WRISO2012014, SCLOR2012014, BCLOR2012014, WSTUK2012014, SSSSM2012014, SEBRS2012014, SSTUK2012014, BSSSM2012014, SLVEA2012014, SKRIL2012014, WIMGW2012014, BVTIG2012014, BSTUK2012014]"
2007033,"[SSTUK2007033, WRISO2007033, SCLOR2007033, WIMGW2007033, SKRIL2007033]"
2002125,"[SSTUK2002125, SDHIG2002125]"


Below we will create a callback `AddSampleIDCB` to remap the `KEY` column to the `SMP_ID` column in each DataFrame.

Remeber that in HELCOM, the `KEY` column has the sample type (S=Sediment, W=Seawater, B=Biota), the laboratory code (e.g., DHIG), followed by an integer sequence. 

If we update the MARIS LUT (dbo_lab.xlsx), to include laboratory codes (i.e. update the ``lab_abb`` column), then the remapping of the  `LAB` and the `AddSampleIDCB` can be much simpler.

In [129]:
# | exports
smp_type_lut = {
    'SEAWATER': 1,
    'BIOTA': 2,
    'SEDIMENT': 3
}

In [130]:
# | exports
class AddSampleIDCB(Callback):
    "Remap `KEY` column to `SMP_ID` in each DataFrame."
    def __init__(self, lut_type: Dict[str, int]):
        self.lut_type = lut_type
        
    def __call__(self, tfm: Transformer):
        for grp in tfm.dfs:
            self._remap_sample_id(tfm.dfs[grp], grp)
    
    def _remap_sample_id(self, df: pd.DataFrame, grp: str):
        """
        Remaps the 'KEY' column to 'SMP_ID' using the provided lookup table.
        Sets 'SMP_ID' to -1 if 'LAB' or 'SEQUENCE' is NaN.
        
        Parameters:
            df (pd.DataFrame): The DataFrame to process.
            grp (str): The group key from the DataFrame dictionary, used to access specific LUT values.
        """
        # Check for NaNs in 'LAB' or 'SEQUENCE' and compute 'SMP_ID' conditionally
        df['SMP_ID'] = np.where(
            df['LAB'].isna() | df['SEQUENCE'].isna(),
            -1,
            str(self.lut_type[grp]) + df['LAB'].astype(str).str.zfill(3) + df['SEQUENCE'].astype(str).str.zfill(7)
        )

        # Convert 'SMP_ID' to integer, handling floating point representations
        df['SMP_ID'] = df['SMP_ID'].apply(lambda x: int(float(x)) if isinstance(x, str) and '.' in x else int(x))

In [131]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                        RemapCB(fn_lut=lut_lab, col_remap='LAB', col_src='LABORATORY', dest_grps=['BIOTA','SEDIMENT','SEAWATER']),
                        AddSampleIDCB(lut_type=smp_type_lut),
                        CompareDfsAndTfmCB(dfs)
                        ])

print(tfm()['SEAWATER']['SMP_ID'].unique())
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


Unmatched value: nan
[12112012003 12112012004 12112012005 ... 13400201806 13400201807
 13400201808]
                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37347
Number of rows removed         0         0         0 



## Add Methods (FOR NEXT VERSION)

lut_method = lambda: pd.read_csv(Path(fname_in) / 'ANALYSIS_METHOD.csv').set_index('METHOD').to_dict()['DESCRIPTION']

The HELCOM dataset includes a look-up table `ANALYSIS_METHOD.csv` which captures the methods used by HELCOM in a description field (free text). Lets review the ANALYSIS METHOD descriptions of HELCOM dataset.


In [ ]:
#| eval: false
analsis_method_df=pd.read_csv(Path(fname_in) / 'ANALYSIS_METHOD.csv')
analsis_method_df.head(3)

,METHOD,COUNTRY,DESCRIPTION
0,BFFG01,6,"Gammaspectrometric analysis with Germanium detectors (p-type HGeLi's and HPGe's and 1 n-type HPGe), with efficiency 20-48% Energy resolution 1.8-2.3 keV at 1.33 MeV (not to in use any more)"
1,BFFG02,6,"Sr-90, a) Y-90 extraction method dried ash and added Y-90 + HCl, Ph adjustment and Y-90 extraction with HDEHP in n-heptane b) Modified version of classic nitric acid method (not to in use any more)"
2,CLOR02,67,"Radiochemical method Radiocaesium separation from seawater samples.134+137Cs was adsorbed on AMP mat, dissolved with NaOH and after purification precipitated as chloroplatinate (Cs2PtCl6).Counting with low background anticoincidence beta counter."


Number of unique ANALYSIS_METHOD DESCRIPTION

In [ ]:
#| eval: false
len(analsis_method_df['DESCRIPTION'].unique())

63

In [ ]:
#| exports
lut_method = lambda: pd.read_csv(Path(fname_in) / 'ANALYSIS_METHOD.csv').set_index('METHOD').to_dict()['DESCRIPTION']

class RemapSedSliceTopBottomCB(Callback):
    "Remap Sediment slice top and bottom to MARIS format."
    def __call__(self, tfm: Transformer):
        "Iterate through all DataFrames in the transformer object and remap sediment slice top and bottom."
        tfm.dfs['SEDIMENT']['TOP'] = tfm.dfs['SEDIMENT']['UPPSLI']
        tfm.dfs['SEDIMENT']['BOTTOM'] = tfm.dfs['SEDIMENT']['LOWSLI']

In [ ]:
#| eval: false
prepmet_lut = pd.read_excel(prepmet_lut_path())
sampmet_lut = pd.read_excel(sampmet_lut_path())
counmet_lut = pd.read_excel(counmet_lut_path())

**DISCUSS** repition of counting method in `counmet_lut`. When should we use each of them?

In [ ]:
#| eval: false
counmet_lut.head(10)

,counmet_id,counmet,code
0,-1,Not applicable,NaN
1,0,Not available,0
2,1,Atomic absorption,AA
3,2,Alpha,ALP
4,3,Alpha ionization chamber spectrometry,ALPI
5,4,Alpha liquid scintillation spectrometry,ALPL
6,5,Alpha semiconductor spectrometry,ALPS
7,6,Alpha total,ALPT
8,7,Accelerator mass spectrometry,AMS
9,8,Beta,BET


## Add slice position (TOP and BOTTOM)

In [ ]:
#| exports
class RemapSedSliceTopBottomCB(Callback):
    "Remap Sediment slice top and bottom to MARIS format."
    def __call__(self, tfm: Transformer):
        "Iterate through all DataFrames in the transformer object and remap sediment slice top and bottom."
        tfm.dfs['SEDIMENT']['TOP'] = tfm.dfs['SEDIMENT']['UPPSLI']
        tfm.dfs['SEDIMENT']['BOTTOM'] = tfm.dfs['SEDIMENT']['LOWSLI']

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[RemapSedSliceTopBottomCB()])
tfm()
print(tfm.dfs['SEDIMENT'][['TOP','BOTTOM']].head())


    TOP  BOTTOM
0  15.0    20.0
1  20.0    27.0
2   0.0     2.0
3   2.0     4.0
4   4.0     6.0


## Add dry weight, wet weight and percentage weight 

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Entries for the ``BASIS`` value of the ``BIOTA`` dataset report a value of `F` which is not consistent with the HELCOM description provided in the metadata. The `GUIDELINES FOR MONITORING OF RADIOACTIVE SUBSTANCES` was obtained from [here](https://metadata.helcom.fi/geonetwork/srv/fin/catalog.search#/metadata/2fdd2d46-0329-40e3-bf96-cb08c7206a24).

:::

Lets take a look at the BIOTA BASIS values:

In [ ]:
#| eval: false
dfs['BIOTA']['BASIS'].unique()

array(['W', nan, 'D', 'F'], dtype=object)

Number of entries for each ``BASIS`` value:

In [ ]:
#| eval: false
dfs['BIOTA']['BASIS'].value_counts()

BASIS
W    11167
D     3634
F       25
Name: count, dtype: int64

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Some entries for ``DW%`` (Dry weight as percentage (%) of fresh weight) are much higher than 100%. Additionally, ``DW%`` is repoted as 0% in some cases.

:::

For BIOTA, the number of entries for ``DW%`` higher than 100%:

In [ ]:
#| eval: false
dfs['BIOTA']['DW%'][dfs['BIOTA']['DW%'] > 100].count()

20

For BIOTA, the number of entries for ``DW%`` equal to 0%:

In [ ]:
#| eval: false
dfs['BIOTA']['DW%'][dfs['BIOTA']['DW%'] == 0].count()


6

For SEDIMENT, the number of entries for ``DW%`` higher than 100%:

In [ ]:
#| eval: false
dfs['SEDIMENT']['DW%'][dfs['SEDIMENT']['DW%'] > 100].count()

621

For SEDIMENT, the number of entries for ``DW%`` equal to 0%:

In [ ]:
#| eval: false
dfs['SEDIMENT']['DW%'][dfs['SEDIMENT']['DW%'] == 0].count()


302

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Several SEDIMENT entries have `DW%` (Dry weight as percentage of fresh weight) values less than 1%. While technically possible, this would indicate samples contained more than 99% water content.

:::

For SEDIMENT, the number of entries for ``DW%`` less than 1% but greater than 0.001%:

In [ ]:
#| eval: false
percent=1
dfs['SEDIMENT']['DW%'][(dfs['SEDIMENT']['DW%'] < percent) & (dfs['SEDIMENT']['DW%'] > 0.001)].count()

24

Lets take a look at the MARIS description of the `percentwt`, `drywt` and `wetwt` variables:

- `percentwt`: Dry weight as ratio of fresh weight, expressed as a decimal .
- `drywt`: Dry weight in grams.
- `wetwt`: Fresh weight in grams.


Lets take a look at the HELCOM dataset, the weight of the sample is not reported for ``SEDIMENT``. However, the percentage dry weight is reported as `DW%`.

In [ ]:
#| eval: false
dfs['SEDIMENT'].columns

Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'ERROR%_kg',
       '< VALUE_Bq/m²', 'VALUE_Bq/m²', 'ERROR%_m²', 'DATE_OF_ENTRY_x',
       'COUNTRY', 'LABORATORY', 'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY',
       'STATION', 'LATITUDE (ddmmmm)', 'LATITUDE (dddddd)',
       'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)', 'DEVICE', 'TDEPTH',
       'UPPSLI', 'LOWSLI', 'AREA', 'SEDI', 'OXIC', 'DW%', 'LOI%',
       'MORS_SUBBASIN', 'HELCOM_SUBBASIN', 'SUM_LINK', 'DATE_OF_ENTRY_y'],
      dtype='object')

The BIOTA dataset reports the weight of the sample as `WEIGHT` and the percentage dry weight as `DW%`. The `BASIS` column describes the basis the value reported

In [ ]:
#| eval: false
dfs['BIOTA'].columns


Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'BASIS',
       'ERROR%', 'NUMBER', 'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY',
       'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY', 'STATION',
       'LATITUDE ddmmmm', 'LATITUDE dddddd', 'LONGITUDE ddmmmm',
       'LONGITUDE dddddd', 'SDEPTH', 'RUBIN', 'BIOTATYPE', 'TISSUE', 'NO',
       'LENGTH', 'WEIGHT', 'DW%', 'LOI%', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN',
       'DATE_OF_ENTRY_y'],
      dtype='object')

In [ ]:
#| exports
class LookupDryWetPercentWeightCB(Callback):
    "Lookup dry-wet ratio and format for MARIS."
    def __call__(self, tfm: Transformer):
        "Iterate through all DataFrames in the transformer object and apply the dry-wet ratio lookup."
        for grp in tfm.dfs.keys():
            if 'DW%' in tfm.dfs[grp].columns:
                self._apply_dry_wet_ratio(tfm.dfs[grp])
            if 'WEIGHT' in tfm.dfs[grp].columns and 'BASIS' in tfm.dfs[grp].columns:
                self._correct_basis(tfm.dfs[grp])
                self._apply_weight(tfm.dfs[grp])

    def _apply_dry_wet_ratio(self, df: pd.DataFrame) -> None:
        "Apply dry-wet ratio conversion and formatting to the given DataFrame."
        df['PERCENTWT'] = df['DW%'] / 100  # Convert percentage to fraction
        df.loc[df['PERCENTWT'] == 0, 'PERCENTWT'] = np.NaN  # Convert 0% to NaN

    def _correct_basis(self, df: pd.DataFrame) -> None:
        "Correct BASIS values. Assuming F = Fresh weight, so F = W"
        df.loc[df['BASIS'] == 'F', 'BASIS'] = 'W'

    def _apply_weight(self, df: pd.DataFrame) -> None:
        "Apply weight conversion and formatting to the given DataFrame."
        dry_condition = df['BASIS'] == 'D'
        wet_condition = df['BASIS'] == 'W'
        
        df.loc[dry_condition, 'DRYWT'] = df['WEIGHT']
        df.loc[dry_condition & df['PERCENTWT'].notna(), 'WETWT'] = df['WEIGHT'] / df['PERCENTWT']
        
        df.loc[wet_condition, 'WETWT'] = df['WEIGHT']
        df.loc[wet_condition & df['PERCENTWT'].notna(), 'DRYWT'] = df['WEIGHT'] * df['PERCENTWT']

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LookupDryWetPercentWeightCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print('BIOTA:', tfm.dfs['BIOTA'][['PERCENTWT','DRYWT','WETWT']].head())
print('SEDIMENT:', tfm.dfs['SEDIMENT']['PERCENTWT'].head())



                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37347
Number of rows removed         0         0         0 

BIOTA:    PERCENTWT      DRYWT  WETWT
0    0.18453  174.93444  948.0
1    0.18453  174.93444  948.0
2    0.18453  174.93444  948.0
3    0.18453  174.93444  948.0
4    0.18458  177.93512  964.0
SEDIMENT: 0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: PERCENTWT, dtype: float64


Note that the dry weight is greater than the wet weight for some entries in the BIOTA dataset due to the DW% being greater than 100%, see above. Lets take a look at the number of entries where this is the case:

In [ ]:
tfm.dfs['BIOTA'][['DRYWT','WETWT']][tfm.dfs['BIOTA']['DRYWT'] > tfm.dfs['BIOTA']['WETWT']].count()

DRYWT    20
WETWT    20
dtype: int64

class ParseCoordinates(Callback):
    """
    Get geographical coordinates from columns expressed in degrees decimal format 
    or from columns in degrees/minutes decimal format where degrees decimal format is missing or zero.
    """
    def __init__(self, 
                 fn_convert_cor: Callable # Function that converts coordinates from degree-minute to decimal degree format
                 ):
        self.fn_convert_cor = fn_convert_cor

    def __call__(self, tfm:Transformer):
        for df in tfm.dfs.values():
            self._format_coordinates(df)

    def _format_coordinates(self, df:pd.DataFrame) -> None:
        coord_cols = self._get_coord_columns(df.columns)
        
        for coord in ['lat', 'lon']:
            decimal_col, minute_col = coord_cols[f'{coord}_d'], coord_cols[f'{coord}_m']
            
            condition = df[decimal_col].isna() | (df[decimal_col] == 0)
            df[coord.upper()] = np.where(condition,
                                 df[minute_col].apply(self._safe_convert),
                                 df[decimal_col])
        
        df.dropna(subset=['LAT', 'LON'], inplace=True)

    def _get_coord_columns(self, columns) -> dict:
        return {
            'lon_d': self._find_coord_column(columns, 'LON', 'dddddd'),
            'lat_d': self._find_coord_column(columns, 'LAT', 'dddddd'),
            'lon_m': self._find_coord_column(columns, 'LON', 'ddmmmm'),
            'lat_m': self._find_coord_column(columns, 'LAT', 'ddmmmm')
        }

    def _find_coord_column(self, columns, coord_type, coord_format) -> str:
        pattern = re.compile(f'{coord_type}.*{coord_format}', re.IGNORECASE)
        matching_columns = [col for col in columns if pattern.search(col)]
        return matching_columns[0] if matching_columns else None

    def _safe_convert(self, value) -> str:
        if pd.isna(value):
            return value
        try:
            return self.fn_convert_cor(value)
        except Exception as e:
            print(f"Error converting value {value}: {e}")
            return value

:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: Column names for geographical coordinates are inconsistent across sample types (biota, sediment, seawater). Sometimes using parentheses, sometimes not.

:::

In [ ]:
#| eval: false
dfs = load_data(fname_in)
for grp in dfs.keys():
    print(f'{grp}: {[col for col in dfs[grp].columns if "LON" in col or "LAT" in col]}')

BIOTA: ['LATITUDE ddmmmm', 'LATITUDE dddddd', 'LONGITUDE ddmmmm', 'LONGITUDE dddddd']
SEAWATER: ['LATITUDE (ddmmmm)', 'LATITUDE (dddddd)', 'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)']
SEDIMENT: ['LATITUDE (ddmmmm)', 'LATITUDE (dddddd)', 'LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)']


:::{.callout-tip}

**FEEDBACK TO DATA PROVIDER**: HELCOM SEAWATER datase includes values of 0 for both latitude and longitude. 

:::

In [ ]:
#| eval: false
def get_invalid_coordinate_df(df, lat_cols, lon_cols):
    invalid_dfs = {}
    
    for lat_col, lon_col in zip(lat_cols, lon_cols):
        # Filter rows where latitude or longitude is NaN or zero
        invalid_df = df[(df[lat_col].isna() | df[lon_col].isna()) | 
                        (df[lat_col] == 0) | (df[lon_col] == 0)]
        
        # Store the invalid DataFrame in the dictionary
        if not invalid_df.empty:
            invalid_dfs[f'{lat_col}_{lon_col}'] = invalid_df

    return invalid_dfs

def print_invalid_coordinates(invalid_dfs, dataset_name):
    for key, invalid_df in invalid_dfs.items():
        print(f'{dataset_name} invalid coordinates for {key}:')
        print(invalid_df.head())

# Define the columns for each dataset
biota_lat_cols = ['LATITUDE ddmmmm', 'LATITUDE dddddd']
biota_lon_cols = ['LONGITUDE ddmmmm', 'LONGITUDE dddddd']

seawater_lat_cols = ['LATITUDE (ddmmmm)', 'LATITUDE (dddddd)']
seawater_lon_cols = ['LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)']

sediment_lat_cols = ['LATITUDE (ddmmmm)', 'LATITUDE (dddddd)']
sediment_lon_cols = ['LONGITUDE (ddmmmm)', 'LONGITUDE (dddddd)']

# Get invalid coordinate DataFrames for each dataset
biota_invalid_dfs = get_invalid_coordinate_df(dfs['BIOTA'], biota_lat_cols, biota_lon_cols)
seawater_invalid_dfs = get_invalid_coordinate_df(dfs['SEAWATER'], seawater_lat_cols, seawater_lon_cols)
sediment_invalid_dfs = get_invalid_coordinate_df(dfs['SEDIMENT'], sediment_lat_cols, sediment_lon_cols)

# Print only non-empty invalid DataFrames
print_invalid_coordinates(biota_invalid_dfs, 'BIOTA')
print_invalid_coordinates(seawater_invalid_dfs, 'SEAWATER')
print_invalid_coordinates(sediment_invalid_dfs, 'SEDIMENT')

SEAWATER invalid coordinates for LATITUDE (dddddd)_LONGITUDE (dddddd):
                KEY NUCLIDE  METHOD < VALUE_Bq/m³  VALUE_Bq/m³  ERROR%_m³  \
19238  WSTUK2015001      H3  STUK04             <        920.0        0.0   
19239  WSTUK2015002      H3  STUK04             <        920.0        0.0   
19240  WSTUK2015003      H3  STUK04             <        920.0        0.0   
19241  WSTUK2015004      H3  STUK04             <        920.0        0.0   
19242  WSTUK2015005      H3  STUK04             <        920.0        0.0   

         DATE_OF_ENTRY_x  COUNTRY LABORATORY  SEQUENCE  ...  \
19238  12/07/16 00:00:00       34       STUK   2015001  ...   
19239  12/07/16 00:00:00       34       STUK   2015002  ...   
19240  12/07/16 00:00:00       34       STUK   2015003  ...   
19241  12/07/16 00:00:00       34       STUK   2015004  ...   
19242  12/07/16 00:00:00       34       STUK   2015005  ...   

      LONGITUDE (ddmmmm)  LONGITUDE (dddddd)  TDEPTH  SDEPTH SALIN  TTEMP  \
19238     

In [ ]:
#| exports
class ParseCoordinates(Callback):
    """
    Get geographical coordinates from columns expressed in degrees decimal format 
    or from columns in degrees/minutes decimal format where degrees decimal format is missing or zero.
    """
    def __init__(self, 
                 fn_convert_cor: Callable # Function that converts coordinates from degree-minute to decimal degree format
                 ):
        self.fn_convert_cor = fn_convert_cor

    def __call__(self, tfm:Transformer):
        for df in tfm.dfs.values():
            self._format_coordinates(df)

    def _format_coordinates(self, df:pd.DataFrame) -> None:
        coord_cols = self._get_coord_columns(df.columns)
        
        for coord in ['lat', 'lon']:
            decimal_col, minute_col = coord_cols[f'{coord}_d'], coord_cols[f'{coord}_m']
            
            condition = df[decimal_col].isna() | (df[decimal_col] == 0)
            df[coord.upper()] = np.where(condition,
                                 df[minute_col].apply(self._safe_convert),
                                 df[decimal_col])
        
        df.dropna(subset=['LAT', 'LON'], inplace=True)

    def _get_coord_columns(self, columns) -> dict:
        return {
            'lon_d': self._find_coord_column(columns, 'LON', 'dddddd'),
            'lat_d': self._find_coord_column(columns, 'LAT', 'dddddd'),
            'lon_m': self._find_coord_column(columns, 'LON', 'ddmmmm'),
            'lat_m': self._find_coord_column(columns, 'LAT', 'ddmmmm')
        }

    def _find_coord_column(self, columns, coord_type, coord_format) -> str:
        pattern = re.compile(f'{coord_type}.*{coord_format}', re.IGNORECASE)
        matching_columns = [col for col in columns if pattern.search(col)]
        return matching_columns[0] if matching_columns else None

    def _safe_convert(self, value) -> str:
        if pd.isna(value):
            return value
        try:
            return self.fn_convert_cor(value)
        except Exception as e:
            print(f"Error converting value {value}: {e}")
            return value

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[                    
                            ParseCoordinates(ddmm_to_dd),
                            CompareDfsAndTfmCB(dfs)
                            ])
tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['BIOTA'][['LAT','LON']])

                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37346
Number of rows removed         0         0         1 

             LAT        LON
0      54.283333  12.316667
1      54.283333  12.316667
2      54.283333  12.316667
3      54.283333  12.316667
4      54.283333  12.316667
...          ...        ...
14888  54.583300  19.000000
14889  54.333300  15.500000
14890  54.333300  15.500000
14891  54.333300  15.500000
14892  54.363900  19.433300

[14893 rows x 2 columns]


Sanitize coordinates drops a row when both longitude & latitude equal 0 or data contains unrealistic longitude & latitude values. Converts longitude & latitude `,` separator to `.` separator."

In [ ]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            ParseCoordinates(ddmm_to_dd),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')
print(tfm.dfs['BIOTA'][['LAT','LON']])


                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14893     20318     37346
Number of rows removed         0         0         1 

             LAT        LON
0      54.283333  12.316667
1      54.283333  12.316667
2      54.283333  12.316667
3      54.283333  12.316667
4      54.283333  12.316667
...          ...        ...
14888  54.583300  19.000000
14889  54.333300  15.500000
14890  54.333300  15.500000
14891  54.333300  15.500000
14892  54.363900  19.433300

[14893 rows x 2 columns]


## Review all callbacks

In [162]:
#| eval: false
dfs = load_data(fname_in)
tfm = Transformer(dfs, cbs=[
                            LowerStripNameCB(col_src='NUCLIDE'),
                            RemapNuclideNameCB(lut_nuclides),
                            ParseTimeCB(),
                            EncodeTimeCB(),
                            SanitizeValueCB(coi_val),       
                            NormalizeUncCB(),
                            RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='TISSUE', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_biogroup_from_biota, col_remap='BIO_GROUP', col_src='SPECIES', dest_grps='BIOTA'),
                            RemapSedimentCB(fn_lut=lut_sediments, replace_lut=sed_replace_lut),
                            RemapUnitCB(),
                            RemapDetectionLimitCB(coi_dl, lut_dl),
                            RemapFiltCB(lut_filtered),
                            RemapCB(fn_lut=lut_lab, col_remap='LAB', col_src='LABORATORY', dest_grps=['BIOTA','SEDIMENT','SEAWATER']),
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetPercentWeightCB(),
                            ParseCoordinates(ddmm_to_dd),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
print(pd.DataFrame.from_dict(tfm.compare_stats) , '\n')


Unmatched SEDI: -99.0
                           BIOTA  SEAWATER  SEDIMENT
Number of rows in dfs      14893     20318     37347
Number of rows in tfm.dfs  14873     20242     37089
Number of rows removed        20        76       258 



In [163]:
grp = 'BIOTA'
print(f'{grp} columns:')
print(tfm.dfs[grp].columns)
print(tfm.dfs[grp].NUCLIDE.unique())

BIOTA columns:
Index(['KEY', 'NUCLIDE', 'METHOD', '< VALUE_Bq/kg', 'VALUE_Bq/kg', 'BASIS',
       'ERROR%', 'NUMBER', 'DATE_OF_ENTRY_x', 'COUNTRY', 'LABORATORY',
       'SEQUENCE', 'DATE', 'YEAR', 'MONTH', 'DAY', 'STATION',
       'LATITUDE ddmmmm', 'LATITUDE dddddd', 'LONGITUDE ddmmmm',
       'LONGITUDE dddddd', 'SDEPTH', 'RUBIN', 'BIOTATYPE', 'TISSUE', 'NO',
       'LENGTH', 'WEIGHT', 'DW%', 'LOI%', 'MORS_SUBBASIN', 'HELCOM_SUBBASIN',
       'DATE_OF_ENTRY_y', 'TIME', 'VALUE', 'UNCERTAINTY', 'SPECIES',
       'BODY_PART', 'BIO_GROUP', 'UNIT', 'DL', 'LAB', 'PERCENTWT', 'DRYWT',
       'WETWT', 'LAT', 'LON'],
      dtype='object')
[31  4  9 33 12 21  6  8 22 10 24 77 17  2 37 41 47 23 11 13 25 16 14 36
 35 29 34 67 63 46 43 42 94 55 50 40 53 87 92 86 15  7 93 85 91 90 51 59
 76 72 54 57]


For instance, lets inspect dropped rows for SEDIMENT:

In [169]:

grp='SEDIMENT' # 'SEAWATER', 'BIOTA' or 'SEDIMENT'
print(f'{grp}, no of dropped rows: {tfm.dfs_dropped[grp].shape[0]}')
view_number=5
print(f'Viewing {view_number} dropped rows for {grp}:')
tfm.dfs_dropped[grp].head(view_number)


SEDIMENT, no of dropped rows: 258
Viewing 5 dropped rows for SEDIMENT:


,KEY,NUCLIDE,METHOD,< VALUE_Bq/kg,VALUE_Bq/kg,ERROR%_kg,< VALUE_Bq/m²,VALUE_Bq/m²,ERROR%_m²,DATE_OF_ENTRY_x,...,LOWSLI,AREA,SEDI,OXIC,DW%,LOI%,MORS_SUBBASIN,HELCOM_SUBBASIN,SUM_LINK,DATE_OF_ENTRY_y
11784,SLREB1998021,SR90,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.0,0.021,55.0,O,NaN,NaN,14.0,14.0,a,NaN
11824,SLVDC1997023,CS137,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,0.021,55.0,O,NaN,NaN,9.0,9.0,a,NaN
11832,SLVDC1997031,CS137,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14.0,0.021,55.0,O,NaN,NaN,9.0,9.0,a,NaN
11841,SLVDC1997040,CS137,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,0.021,55.0,O,NaN,NaN,9.0,9.0,a,NaN
11849,SLVDC1998011,CS137,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,0.021,55.0,O,NaN,NaN,14.0,14.0,a,NaN


### Example change logs

In [164]:
#| eval: false
dfs = load_data(fname_in)

tfm = Transformer(dfs, cbs=[
                            LowerStripNameCB(col_src='NUCLIDE'),
                            RemapNuclideNameCB(lut_nuclides),
                            ParseTimeCB(),
                            EncodeTimeCB(),
                            SanitizeValueCB(coi_val),       
                            NormalizeUncCB(),
                            RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='TISSUE', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_biogroup_from_biota, col_remap='BIO_GROUP', col_src='SPECIES', dest_grps='BIOTA'),
                            RemapSedimentCB(fn_lut=lut_sediments, replace_lut=sed_replace_lut),
                            RemapUnitCB(),
                            RemapDetectionLimitCB(coi_dl, lut_dl),
                            RemapFiltCB(lut_filtered),
                            RemapCB(fn_lut=lut_lab, col_remap='LAB', col_src='LABORATORY', dest_grps=['BIOTA','SEDIMENT','SEAWATER']),
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetPercentWeightCB(),
                            ParseCoordinates(ddmm_to_dd),
                            SanitizeLonLatCB(),
                            CompareDfsAndTfmCB(dfs)
                            ])

tfm()
tfm.logs

Unmatched SEDI: -99.0


["Convert values from 'NUCLIDE' to lowercase, strip spaces, and store in 'None'.",
 'Remap data provider nuclide names to MARIS nuclide names.',
 'Parse and standardize time information in the dataframe.',
 'Encode time as seconds since epoch.',
 'Sanitize value/measurement by removing blank entries and populating `value` column.',
 'Convert from relative error % to standard uncertainty.',
 "Remap values from 'RUBIN' to 'SPECIES' for groups: B, I, O, T, A.",
 "Remap values from 'TISSUE' to 'BODY_PART' for groups: B, I, O, T, A.",
 "Remap values from 'SPECIES' to 'BIO_GROUP' for groups: B, I, O, T, A.",
 'Update sediment id based on MARIS species LUT (dbo_sedtype.xlsx).',
 'Set the `unit` id column in the DataFrames based on a lookup table.',
 'Remap value type to MARIS format.',
 'Lookup FILT value in dataframe using the lookup table.',
 "Remap values from 'LABORATORY' to 'LAB' for groups: BIOTA, SEDIMENT, SEAWATER.",
 'Remap Sediment slice top and bottom to MARIS format.',
 'Lookup dr

In [ ]:
#| exports
def get_attrs(
    tfm: Transformer, # Transformer object
    zotero_key: str, # Zotero dataset record key
    kw: list = kw # List of keywords
    ) -> dict: # Global attributes
    "Retrieve all global attributes."
    return GlobAttrsFeeder(tfm.dfs, cbs=[
        BboxCB(),
        DepthRangeCB(),
        TimeRangeCB(),
        ZoteroCB(zotero_key, cfg=cfg()),
        KeyValuePairCB('keywords', ', '.join(kw)),
        KeyValuePairCB('publisher_postprocess_logs', ', '.join(tfm.logs))
        ])()

In [ ]:
#| eval: false
get_attrs(tfm, zotero_key=zotero_key, kw=kw)

{'geospatial_lat_min': '31.17',
 'geospatial_lat_max': '65.75',
 'geospatial_lon_min': '9.6333',
 'geospatial_lon_max': '53.5',
 'geospatial_bounds': 'POLYGON ((9.6333 53.5, 31.17 53.5, 31.17 65.75, 9.6333 65.75, 9.6333 53.5))',
 'time_coverage_start': '1984-01-10T00:00:00',
 'time_coverage_end': '2018-12-14T00:00:00',
 'title': 'Environmental database - Helsinki Commission Monitoring of Radioactive Substances',
 'summary': 'MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.\n\nThe database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.\n\nThe database is updated and quality assured annua

### <a name="encoding-netcdf"></a>Encoding NetCDF

In [ ]:
#| exports
def encode(
    fname_in: str, # Input file name
    fname_out_nc: str, # Output file name
    **kwargs # Additional arguments
    ) -> None:
    "Encode data to NetCDF."
    dfs = load_data(fname_in)
    tfm = Transformer(dfs, cbs=[
                            LowerStripNameCB(col_src='NUCLIDE'),
                            RemapNuclideNameCB(lut_nuclides),
                            ParseTimeCB(),
                            EncodeTimeCB(),
                            SanitizeValueCB(coi_val),       
                            NormalizeUncCB(),
                            RemapCB(fn_lut=lut_biota, col_remap='SPECIES', col_src='RUBIN', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_tissues, col_remap='BODY_PART', col_src='TISSUE', dest_grps='BIOTA'),
                            RemapCB(fn_lut=lut_biogroup_from_biota, col_remap='BIO_GROUP', col_src='SPECIES', dest_grps='BIOTA'),
                            RemapSedimentCB(fn_lut=lut_sediments, replace_lut=sed_replace_lut),
                            RemapUnitCB(),
                            RemapDetectionLimitCB(coi_dl, lut_dl),
                            RemapFiltCB(lut_filtered),
                            RemapSedSliceTopBottomCB(),
                            LookupDryWetPercentWeightCB(),
                            ParseCoordinates(ddmm_to_dd),
                            SanitizeLonLatCB(),
                            ])
    tfm()
    encoder = NetCDFEncoder(tfm.dfs, 
                            dest_fname=fname_out_nc, 
                            global_attrs=get_attrs(tfm, zotero_key=zotero_key, kw=kw),
                            verbose=kwargs.get('verbose', False),
                           )
    encoder.encode()

In [ ]:
#| eval: false
encode(fname_in, fname_out_nc, verbose=True)

Unmatched SEDI: -99.0
--------------------------------------------------------------------------------
Group: biota, Variable: lon
--------------------------------------------------------------------------------
Group: biota, Variable: lat
--------------------------------------------------------------------------------
Group: biota, Variable: time
--------------------------------------------------------------------------------
Group: biota, Variable: nuclide
--------------------------------------------------------------------------------
Group: biota, Variable: value
--------------------------------------------------------------------------------
Group: biota, Variable: unit
--------------------------------------------------------------------------------
Group: biota, Variable: dl
--------------------------------------------------------------------------------
Group: biota, Variable: bio_group
--------------------------------------------------------------------------------
Group: biota

## NetCDF QA  

First lets review the general properties of the NetCDF file:

In [ ]:
#| eval: false
properties=get_netcdf_properties(fname_out_nc)
for key, val in properties.items():
    if isinstance(val, dict):
        print(f"{key}:")
        for sub_key, sub_val in val.items():
            print(f"  {sub_key}: {sub_val}")
    else:
        print(f"{key}: {val}")

file_size_bytes: 669457
file_format: NETCDF4
groups: ['biota', 'seawater', 'sediment']
global_attributes:
  id: TBD
  title: Environmental database - Helsinki Commission Monitoring of Radioactive Substances
  summary: MORS Environment database has been used to collate data resulting from monitoring of environmental radioactivity in the Baltic Sea based on HELCOM Recommendation 26/3.

The database is structured according to HELCOM Guidelines on Monitoring of Radioactive Substances (https://www.helcom.fi/wp-content/uploads/2019/08/Guidelines-for-Monitoring-of-Radioactive-Substances.pdf), which specifies reporting format, database structure, data types and obligatory parameters used for reporting data under Recommendation 26/3.

The database is updated and quality assured annually by HELCOM MORS EG.
  keywords: oceanography, Earth Science > Oceans > Ocean Chemistry> Radionuclides, Earth Science > Human Dimensions > Environmental Impacts > Nuclear Radiation Exposure, Earth Science > Oceans

:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 
Update TBD values. 
The publisher_postprocess_logs may include ',' in the string. Can we review how the publisher_postprocess_logs are encoded? Would a dictionary be better?
:::

:::{.callout-tip}

**FEEDBACK FOR NEXT VERSION**: 
Enums (LUTS) should be encoded in the NetCDF file. 
:::

Review the publisher_postprocess_logs.

In [ ]:
#| eval: false
print(properties['global_attributes']['publisher_postprocess_logs'])

Convert values from 'NUCLIDE' to lowercase, strip spaces, and store in 'None'., Remap data provider nuclide names to MARIS nuclide names., Parse and standardize time information in the dataframe., Encode time as seconds since epoch., Sanitize value/measurement by removing blank entries and populating `value` column., Convert from relative error % to standard uncertainty., Remap values from 'RUBIN' to 'SPECIES' for groups: B, I, O, T, A., Remap values from 'TISSUE' to 'BODY_PART' for groups: B, I, O, T, A., Remap values from 'SPECIES' to 'BIO_GROUP' for groups: B, I, O, T, A., Update sediment id based on MARIS species LUT (dbo_sedtype.xlsx)., Set the `unit` id column in the DataFrames based on a lookup table., Remap value type to MARIS format., Lookup FILT value in dataframe using the lookup table., Remap Sediment slice top and bottom to MARIS format., Lookup dry-wet ratio and format for MARIS., 
    Get geographical coordinates from columns expressed in degrees decimal format 
    or f

Now lets review the properties of the groups in the NetCDF file:

In [ ]:
#| eval: false
properties = get_netcdf_group_properties(fname_out_nc)

for key, val in properties.items():
    if isinstance(val, dict):
        print(f"{key}:")
        for sub_key, sub_val in val.items():
            print(f"  {sub_key}: {sub_val}")
    else:
        print(f"{key}: {val}")

biota:
  variables: ['lon', 'lat', 'time', 'nuclide', 'value', 'unit', 'dl', 'bio_group', 'species', 'body_part', 'drywt', 'wetwt']
  dimensions: {'id': 14873}
  attributes: {}
seawater:
  variables: ['lon', 'lat', 'time', 'nuclide', 'value', 'unit', 'dl', 'filt']
  dimensions: {'id': 20242}
  attributes: {}
sediment:
  variables: ['lon', 'lat', 'time', 'area', 'nuclide', 'value', 'unit', 'dl', 'sed_type', 'top', 'bottom']
  dimensions: {'id': 37089}
  attributes: {}


Lets review all variable attributes for the groups of the NetCDF file:

In [ ]:
#| eval: false
df_var_prop=get_netcdf_variable_properties(fname_out_nc, as_df=True).T
df_var_prop

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
group,biota,biota,biota,biota,biota,biota,biota,biota,biota,biota,...,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment,sediment
variable,lon,lat,time,nuclide,value,unit,dl,bio_group,species,body_part,...,lat,time,area,nuclide,value,unit,dl,sed_type,top,bottom
dimensions_id,"('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)",...,"('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)","('id',)"
dimensions_size,"(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)","(14873,)",...,"(37089,)","(37089,)","(37089,)","(37089,)","(37089,)","(37089,)","(37089,)","(37089,)","(37089,)","(37089,)"
data_type,<f4,<f4,<u8,<i8,<f4,<i8,<i8,<i8,<i8,<i8,...,<f4,<u8,<i8,<i8,<f4,<i8,<i8,<i8,<f4,<f4
attr_long_name,Measurement longitude,Measurement latitude,Time of measurement,Nuclide,Activity,Unit,Detection limit,Biota group,Species,Body part,...,Measurement latitude,Time of measurement,Marine area/region id,Nuclide,Activity,Unit,Detection limit,Sediment type,Top depth of sediment layer,Bottom depth of sediment layer
attr_standard_name,longitude,latitude,time,nuclide,activity,unit,detection_limit,biota_group_tbd,species,body_part_tbd,...,latitude,time,area_id,nuclide,activity,unit,detection_limit,sediment_type_tbd,top_depth_of_sediment_layer_tbd,bottom_depth_of_sediment_layer_tbd
attr_units,degrees_east,degrees_north,seconds since 1970-01-01 00:00:00.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,degrees_north,seconds since 1970-01-01 00:00:00.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attr_time_origin,NaN,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1970-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attr_time_zone,NaN,NaN,UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,UTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lets convert the NetCDF file to a dictionary of DataFrames:

In [ ]:
#| eval: false
dfs=nc_to_dfs(fname_out_nc)

Lets review the biota data:

In [ ]:
#| eval: false
nc_dfs_biota=dfs['BIOTA']
nc_dfs_biota

,lon,lat,time,nuclide,value,unit,dl,bio_group,species,body_part,drywt,wetwt
0,12.316667,54.283333,2012-09-23,31,0.010140,5,2,4,99,52,174.934433,948.0
1,12.316667,54.283333,2012-09-23,4,135.300003,5,1,4,99,52,174.934433,948.0
2,12.316667,54.283333,2012-09-23,9,0.013980,5,2,4,99,52,174.934433,948.0
3,12.316667,54.283333,2012-09-23,33,4.338000,5,1,4,99,52,174.934433,948.0
4,12.316667,54.283333,2012-09-23,31,0.009614,5,2,4,99,52,177.935120,964.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14868,19.000000,54.583302,2018-02-26,53,0.043000,5,1,4,191,3,120.000000,500.0
14869,15.500000,54.333302,2018-02-13,4,98.000000,5,1,4,191,52,112.500000,500.0
14870,15.500000,54.333302,2018-02-13,33,3.690000,5,1,4,191,52,112.500000,500.0
14871,15.500000,54.333302,2018-02-13,53,0.049000,5,1,4,191,52,112.500000,500.0


Lets review the sediment data:

In [ ]:
#| eval: false
nc_dfs_sediment=dfs['SEDIMENT']
nc_dfs_sediment


,lon,lat,time,area,nuclide,value,unit,dl,sed_type,top,bottom
0,24.0000,59.666698,2012-06-17,0,53,35.00,4,1,0,15.0,20.0
1,24.0000,59.666698,2012-06-17,0,53,36.00,4,1,0,20.0,27.0
2,28.8433,59.860001,2012-08-10,0,53,38.00,4,1,0,0.0,2.0
3,28.8433,59.860001,2012-08-10,0,53,36.00,4,1,0,2.0,4.0
4,28.8433,59.860001,2012-08-10,0,53,30.00,4,1,0,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...
37084,21.0830,59.035999,2016-06-09,0,33,1.20,4,1,50,18.0,20.0
37085,21.0830,59.035999,2016-06-09,0,33,0.79,4,1,50,20.0,22.0
37086,19.7297,61.066700,2016-05-29,0,33,512.00,4,1,59,0.0,2.0
37087,19.7297,61.066700,2016-05-29,0,33,527.00,4,1,51,2.0,4.0


Lets review the seawater data:

In [ ]:
#| eval: false
nc_dfs_seawater=dfs['SEAWATER']
nc_dfs_seawater

,lon,lat,time,nuclide,value,unit,dl,filt
0,29.333300,60.083302,2012-05-23,33,5.300000,1,1,0
1,29.333300,60.083302,2012-05-23,33,19.900000,1,1,0
2,23.150000,59.433300,2012-06-17,33,25.500000,1,1,0
3,27.983299,60.250000,2012-05-24,33,17.000000,1,1,0
4,27.983299,60.250000,2012-05-24,33,22.200001,1,1,0
...,...,...,...,...,...,...,...,...
20237,14.200000,54.006802,2015-06-22,12,6.600000,1,1,1
20238,14.667200,54.499500,2015-06-23,12,6.900000,1,1,1
20239,14.334200,54.750500,2015-06-23,12,6.800000,1,1,1
20240,13.500200,54.916500,2015-06-24,12,7.300000,1,1,1


## Open Refine Decoder (WIP)

Currently, the processing of MARIS data to the master dataset is done in OpenRefine. A standardised CSV is processed in OpenRefine and exported to the MARIS database. 

A decoder is needed to convert the NetCDF file format to the standardised CSV format. 